# **Preparation**

``note: jalankan semua code ini & pastikan pakai GPU biar cepet``

In [ ]:
# Benerin / cek data test apakah include untuk training ato nggak

## **Import Library**

In [ ]:
# Install
!pip install PyDOE
!pip install tensorflow==1.15

In [ ]:
# Import
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import random
import scipy.io
import tensorflow as tf
import time
import timeit
import math as m
import scipy.interpolate

from mpl_toolkits.axes_grid1 import make_axes_locatable
from pyDOE import lhs


In [ ]:
#! sudo apt-get install texlive-latex-recommended 
#! sudo apt install texlive-latex-extra
#! sudo apt install dvipng
#! sudo apt install cm-super

import matplotlib.pyplot as plt
import numpy as np
#from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
#rc('text', usetex=True)

## **Class & Functions**

In [ ]:
np.finfo(float).eps

### **PINN Class**

In [ ]:
class PinnVP:

    def __init__(self, data, params, bc_type, problem_type, adaptive_model=False, improved_model=False, 
                 exist_model=False, mini_batches=False, file_dir=""):
        # Initialize & Unpack Input data
        self.adaptive_model = adaptive_model
        self.improved_model = improved_model
        self.mini_batches = mini_batches
        self.unpack(data, params)
        self.initialize_variables()
        self.bc_type = bc_type 
        self.problem_type = problem_type

        # Initialize Neural Network Computational Graph
        # Weight & biases
        if exist_model:
            print("Loading NN parameters ...")
            self.weights, self.biases = self.load_model(file_dir)
        else:
            self.weights, self.biases = self.initialize_network()

        # Initialize encoder weights & biases
        if self.improved_model == improved_model:
            self.encoder_weights_1 = self.xavier_init([2, self.layers[1]])
            self.encoder_biases_1 = self.xavier_init([1, self.layers[1]])

            self.encoder_weights_2 = self.xavier_init([2, self.layers[1]])
            self.encoder_biases_2 = self.xavier_init([1, self.layers[1]])

        # Placeholder & Graph
        # Placeholder (where we put input)
        self.initialize_placeholders()

        # Computational graph of Physics-Informed
        self.graph_network()

        # Computational graph of loss
        self.graph_loss()

        # Optimizers
        self.initialize_optimizers()

        # Initialize adaptive weights
        if self.adaptive_model:
            self.initialize_adaptive_weights()

        # Session
        self.initialize_session()

    def callback_inverse(self, loss_test, loss_total, loss_collo, loss_meu, loss_init, lambda_1):
        self.count += 1
        self.loss_test_log.append(loss_test)
        self.loss_total_log.append(loss_total)
        self.loss_collo_log.append(loss_collo)
        self.loss_meu_log.append(loss_meu)
        self.loss_init_log.append(loss_init)
        self.lambda_1_log.append(lambda_1)

        if self.count % self.verboses_newton == 0:    
           print("iter: %d, Loss Test: %.4e, Loss Total: %.4e, Loss Collo: %.4e, Loss Measurement: %.4e, Loss Init: %.4e, lambda_1: %.3f" %
                 (self.count, loss_test, loss_total, loss_collo, loss_meu, loss_init, lambda_1[0]))
    
    def callback(self, loss_test, loss_total, loss_collo, loss_meu, loss_init):
        self.count += 1
        self.loss_test_log.append(loss_test)
        self.loss_total_log.append(loss_total)
        self.loss_collo_log.append(loss_collo)
        self.loss_meu_log.append(loss_meu)
        self.loss_init_log.append(loss_init)
        
        if self.count % self.verboses_newton == 0:    
           print("iter: %d, Loss Test: %.4e, Loss Total: %.4e, Loss Collo: %.4e, Loss Measurement: %.4e, Loss Init: %.4e" %
                 (self.count, loss_test, loss_total, loss_collo, loss_meu, loss_init))

        

    def normalize_input_data(self):
        # Normalize data
        x_c = self.normalize_data(data=self.x_c, axis="x")
        # x_c = self.x_c
        y_c = self.normalize_data(data=self.y_c, axis="y")
#         x_inlet = self.normalize_data(data=self.x_inlet, axis="x")
        # x_inlet = self.x_inlet
#         y_inlet = self.normalize_data(data=self.y_inlet, axis="y")
        x_meu = self.normalize_data(data=self.x_meu, axis="x")
        # x_outlet = self.x_outlet
        y_meu = self.normalize_data(data=self.y_meu, axis="y")
        x_wall = self.normalize_data(data=self.x_wall, axis="x")
        # x_wall = self.x_wall
        y_wall = self.normalize_data(data=self.y_wall, axis="y")
        x_test = self.normalize_data(data=self.x_test, axis="x")
        # x_test = self.x_test
        y_test = self.normalize_data(data=self.y_test, axis="y")

        return x_c, y_c, x_meu, y_meu, x_wall, y_wall, x_test, y_test

    def fetch_minibatch(self, x, y):
        mini_batches = []
        batch_size = self.batch_size
        data = np.hstack((x, y))
        np.random.shuffle(data)
        n_mini_batches = data.shape[0] // batch_size

        i = 0
        for i in range(n_mini_batches):
            mini_batch = data[i*batch_size:(i+1)*batch_size, :]
            x_mini = mini_batch[:, :-1]
            y_mini = mini_batch[:, -1].reshape((-1, 1))
            mini_batches.append(np.hstack((x_mini, y_mini)))

        return mini_batches

    def create_input_dict(self):
        # Normalize data
        x_c, y_c, x_inlet, y_inlet, x_outlet, y_outlet, x_wall, y_wall, x_test, y_test = self.normalize_input_data()

        # Mini Batches
        if self.mini_batches:
            tf_dict_batches = []
            mini_batches_input = self.fetch_minibatch(x=x_c, y=y_c)

            for mini_batch in mini_batches_input:
                # unpack
                x_mini = mini_batch[:, 0]
                y_mini = mini_batch[:, 1]

                n = len(x_mini)

                x_mini = x_mini.reshape(n, 1)
                y_mini = y_mini.reshape(n, 1)

                if self.adaptive_model:
                    tf_dict = {self.x_c_tf: x_mini, self.y_c_tf: y_mini,
                               self.x_inlet_tf: x_inlet, self.y_inlet_tf: y_inlet,              # inlet data (pts)
                               self.u_inlet_tf: self.u_inlet, self.v_inlet_tf: self.v_inlet,    # inlet data (velocity)
                               self.x_outlet_tf: x_outlet, self.y_outlet_tf: y_outlet,          # outlet data
                               self.x_wall_tf: x_wall, self.y_wall_tf: y_wall,                  # wall data
                               self.u_wall_tf: self.u_wall, self.v_wall_tf: self.v_wall,        
                               self.x_test_tf: x_test, self.y_test_tf: y_test,                  # test data
                               self.u_test_tf: self.u_test,
                               self.coef_bc_tf: self.coef_bc_val}
                else:
                    tf_dict = {self.x_c_tf: x_mini, self.y_c_tf: y_mini,
                               self.x_inlet_tf: x_inlet, self.y_inlet_tf: y_inlet,              # inlet data (pts)
                               self.u_inlet_tf: self.u_inlet, self.v_inlet_tf: self.v_inlet,    # inlet data (velocity)
                               self.x_outlet_tf: x_outlet, self.y_outlet_tf: y_outlet,          # outlet data
                               self.x_wall_tf: x_wall, self.y_wall_tf: y_wall,                  # wall data
                               self.u_wall_tf: self.u_wall, self.v_wall_tf: self.v_wall,        
                               self.x_test_tf: x_test, self.y_test_tf: y_test,                  # test data
                               self.u_test_tf: self.u_test}

                # add to tf dict batches
                tf_dict_batches.append(tf_dict)

            return tf_dict_batches

        else:
            if self.adaptive_model:
                tf_dict = {self.x_c_tf: x_c, self.y_c_tf: y_c,
                           self.x_inlet_tf: x_inlet, self.y_inlet_tf: y_inlet,              # inlet data (pts)
                           self.u_inlet_tf: self.u_inlet, 
                          #  self.v_inlet_tf: self.v_inlet,    # inlet data (velocity)
                           self.x_outlet_tf: x_outlet, self.y_outlet_tf: y_outlet, 
                           self.u_outlet_tf: self.u_outlet, # outlet data
                           self.x_wall_tf: x_wall, self.y_wall_tf: y_wall,                  # wall data
                           self.u_wall_tf: self.u_wall, 
                          #  self.v_wall_tf: self.v_wall,        
                           self.x_test_tf: x_test, self.y_test_tf: y_test,                  # test data
                           self.u_test_tf: self.u_test,
                           self.coef_bc_tf: self.coef_bc_val}
            else:
                tf_dict = {self.x_c_tf: x_c, self.y_c_tf: y_c,
                            self.x_inlet_tf: x_inlet, self.y_inlet_tf: y_inlet,              # inlet data (pts)
                            self.u_inlet_tf: self.u_inlet, 
                            # self.v_inlet_tf: self.v_inlet,    # inlet data (velocity)
                            self.x_outlet_tf: x_outlet, self.y_outlet_tf: y_outlet,          # outlet data
                            self.u_outlet_tf: self.u_outlet,
                            self.x_wall_tf: x_wall, self.y_wall_tf: y_wall,                  # wall data
                            self.u_wall_tf: self.u_wall, 
                            # self.v_wall_tf: self.v_wall,        
                            self.x_test_tf: x_test, self.y_test_tf: y_test,                  # test data
                            # self.x_tf: self.x,
                            self.u_test_tf: self.u_test}
                            # self.rho_tf: self.RHO, self.cp_tf: self.cp, self.k_tf: self.k}

            return tf_dict
            
    def fit_adam(self):
        # Change flag
        self.adam_started = True

        # Start iteration
        print("START TRAINING ADAM")
        print("-------------------------------------------------------------")
        start_time = timeit.default_timer()

        for epoch in range(self.epochs):
            # Create dictionary
            if self.mini_batches:
                tf_dicts = self.create_input_dict()
            else:
                tf_dict = self.create_input_dict()

            # Run Adam optimizer
            if self.mini_batches:
                for tf_dict in tf_dicts:
                    self.sess.run(self.train_op_adam, tf_dict)
            else:
                self.sess.run(self.train_op_adam, tf_dict)

            # Adaptive modeling
            if self.adaptive_model:
                if epoch % self.verboses_adaptive == 0:
                    coef_bc_val = self.sess.run(self.coef_bc, tf_dict)
                    self.coef_bc_val = coef_bc_val*(1.0-self.beta) + self.beta*self.coef_bc_val
                    self.coef_bc_log.append(self.coef_bc_val)

            # Print
            if epoch % self.verboses_adam == 0:
                # Calculate loss
                loss_total_ = self.sess.run(self.loss_total, tf_dict)
                loss_collo_ = self.sess.run(self.loss_collo, tf_dict)
                loss_bound_ = self.sess.run(self.loss_bound, tf_dict)
                loss_init_ = self.sess.run(self.loss_init, tf_dict)
                loss_test_ = self.sess.run(self.loss_test, tf_dict)

                if self.bc_type == 'Neumann':
                   grad_in = self.sess.run(self.grad_x_inlet,tf_dict)
                   grad_out = self.sess.run(self.grad_x_outlet,tf_dict)

                # Save loss
                self.loss_total_log.append(loss_total_)
                self.loss_collo_log.append(loss_collo_)
                self.loss_bound_log.append(loss_bound_)
                self.loss_init_log.append(loss_init_)
                self.loss_test_log.append(loss_test_)

                if self.problem_type == "Inverse":
                   lambda_1_value = self.sess.run(self.lambda_1)
                   self.lambda_1_log.append(lambda_1_value)

                   elapsed = timeit.default_timer() - start_time
                   print("iter: %d, Loss Test: %.4e, Loss Total: %.4e, Loss Collo: %.4e, Loss Boundary: %.4e, Loss Init: %.4e, lambda_1: %.3f, time: %.2fs" %
                         (epoch+self.last_adam_iter, loss_test_, loss_total_, loss_collo_, loss_bound_, loss_init_, lambda_1_value[0], elapsed))
                else:
                   elapsed = timeit.default_timer() - start_time
                   print("iter: %d, Loss Test: %.4e, Loss Total: %.4e, Loss Collo: %.4e, Loss Boundary: %.4e, Loss Init: %.4e, time: %.2fs" %
                         (epoch+self.last_adam_iter, loss_test_, loss_total_, loss_collo_, loss_bound_, loss_init_, elapsed))

                
                if self.bc_type == "Neumann":
                  print(np.concatenate((grad_in, grad_out), axis=1))
                
                if self.adaptive_model:
                    print("boundary weights coefficients: {:.4f}".format(self.coef_bc_val))

                start_time = timeit.default_timer()

            if epoch % self.saver == 0:
                self.save_model(f"model-{epoch}.pickle")
                self.save_loss(f"model-loss-{epoch}.csv")
                print("model & loss saved")

        self.last_adam_iter += self.epochs
        self.lr_init = self.coef_bc_val

    def fit_newton(self):
        # Change flag
        self.newton_started = True
        self.count += self.last_adam_iter

        # Normalize data
        (x_c, y_c, 
#          x_inlet, y_inlet, 
         x_meu, y_meu, 
         x_wall, y_wall, 
         x_test, y_test) = self.normalize_input_data()

        # Create dictionary
        tf_dict = {self.x_c_tf: x_c, self.y_c_tf: y_c,                              # collocation data
#                    self.x_inlet_tf: x_inlet, self.y_inlet_tf: y_inlet,              # inlet data (pts)
#                    self.u_inlet_tf: self.u_inlet, 
                  #  self.v_inlet_tf: self.v_inlet,    # inlet data (velocity)
                   self.x_meu_tf: x_meu, self.y_meu_tf: y_meu,          # outlet data
                   self.u_meu_tf: self.u_meu,
                   self.x_wall_tf: x_wall, self.y_wall_tf: y_wall,                  # wall data
                   self.u_wall_tf: self.u_wall, 
                  #  self.v_wall_tf: self.v_wall,        
                   self.x_test_tf: x_test, self.y_test_tf: y_test,                  # test data
                  #  self.x_tf:self.x,
                   self.u_test_tf:self.u_test, self.coef_bc_tf:self.coef_bc_val}
                  #  self.rho_tf: self.RHO, self.cp_tf: self.cp, self.k_tf: self.k}

        # Optimize
        if self.problem_type == "Inverse":
           self.train_op_newton.minimize(self.sess,
                                         feed_dict=tf_dict,
                                         fetches=[self.loss_test, self.loss_total, 
                                                 self.loss_collo, self.loss_meu, self.loss_initial,
                                                 self.lambda_1],
                                         loss_callback=self.callback_inverse)
        else:
           self.train_op_newton.minimize(self.sess,
                                          feed_dict=tf_dict,
                                          fetches=[self.loss_test, self.loss_total, 
                                                  self.loss_collo, self.loss_meu, self.loss_initial],
                                          loss_callback=self.callback)
        
        # Return learning rate
        self.lr_init = self.lr_tf
    
    def graph_loss(self):
        # Test
        self.loss_test = tf.math.sqrt(tf.reduce_mean(tf.square(self.u_test_tf - self.u_test_pred))) #/tf.reduce_mean(tf.square(self.u_test_tf)))
        # self.loss_test = tf.math.sqrt(tf.reduce_mean(tf.square(self.u_test_tf - self.u_test_pred)))

        # Collocation points
        self.loss_collo = tf.reduce_mean(tf.square(self.f_pred_u)) \
                          # + tf.reduce_mean(tf.square(self.f_pred_v))
        if self.bc_type == 'Dirichlet':
            # Boundary
            # Inlet: u & v to dirichlet conditions
#             self.loss_inlet = tf.reduce_mean(tf.square(self.u_inlet_pred-self.u_inlet_tf)) 
                          # + tf.reduce_mean(tf.square(self.v_inlet_pred-self.v_inlet_tf))
        

            # # Outlet: p to dirichlet conditions
            # self.loss_outlet = tf.reduce_mean(tf.square(self.p_outlet_pred))
            self.loss_meu = tf.reduce_mean(tf.square(self.u_meu_pred-self.u_meu_tf))
            
        elif self.bc_type == 'Neumann':
            
            
            T_out = self.net_dnn(self.x_outlet_tf, self.y_outlet_tf)
            T_in = self.net_dnn(self.x_inlet_tf, self.y_inlet_tf)
            self.grad_x_outlet = tf.gradients(T_out, self.x_outlet_tf)[0] / self.sigma_x
            self.grad_x_inlet = tf.gradients(T_in, self.x_inlet_tf)[0] / self.sigma_x

            k = self.k
            k_out = (T_out - k[0,0])/(k[1,0] - k[0,0])*(k[1,1] - k[0,1]) + k[0,1]
            k_in = (T_in - k[0,0])/(k[1,0] - k[0,0])*(k[1,1] - k[0,1]) + k[0,1]

            self.loss_inlet = tf.reduce_mean(tf.square(k_in*self.grad_x_inlet-self.u_inlet_tf)) 
            self.loss_outlet = tf.reduce_mean(tf.square(k_out*self.grad_x_outlet-self.u_outlet_tf))

        # Wall: u & v to dirichlet conditions
        self.loss_initial = tf.reduce_mean(tf.square(self.u_wall_pred-self.u_wall_tf)) 
                        #  + tf.reduce_mean(tf.square(self.v_wall_pred-self.v_wall_tf))
        
#         if self.adaptive_model:
#             self.loss_bound = self.coef_bc_tf*(self.loss_inlet + self.loss_outlet)
#             self.loss_init = self.coef_bc_tf*(self.loss_initial)
#         else:
#             self.loss_bound = self.coef_bc_val*(self.loss_inlet + self.loss_outlet)
#             self.loss_init = self.coef_bc_val*self.loss_initial

        # Total loss
        self.loss_total = self.loss_collo + self.loss_meu + self.loss_initial

    def graph_network(self):
        # Test data
        # if self.problem_type == 'Inverse':
        #    (self.u_test_pred, self.n_test_pred) = self.net_dnn(self.x_test_tf, self.y_test_tf)
        #    (self.u_pred, self.n_pred) = self.net_dnn(self.x_tf, self.y_tf)
        #    (self.f_pred_u, ) = self.net_physics(self.x_c_tf, self.y_c_tf)
        #    (self.u_inlet_pred) = self.net_dnn(self.x_inlet_tf, self.y_inlet_tf)
        #    (self.u_outlet_pred, self.n_outlet) = self.net_dnn(self.x_outlet_tf, self.y_outlet_tf)
        #    (self.u_wall_pred, self.n_wall_pred) = self.net_dnn(self.x_wall_tf, self.y_wall_tf)
        # (self.u_test_pred, self.v_test_pred, self.p_test_pred) = self.net_dnn(self.x_test_tf, self.y_test_tf)
        (self.u_test_pred) = self.net_dnn(self.x_test_tf, self.y_test_tf)
        
        # Predict data
        # (self.u_pred, self.v_pred, self.p_pred) = self.net_dnn(self.x_tf, self.y_tf)
        (self.u_pred) = self.net_dnn(self.x_tf, self.y_tf)

        # Physics Training
        # Collocation points
        # (self.f_pred_u, self.f_pred_v) = self.net_physics(self.x_c_tf, self.y_c_tf)
        (self.f_pred_u) = self.net_physics(self.x_c_tf, self.y_c_tf)

        # Inlet: u & v
        # (self.u_inlet_pred, self.v_inlet_pred, _) = self.net_dnn(self.x_inlet_tf, self.y_inlet_tf)
#         (self.u_inlet_pred) = self.net_dnn(self.x_inlet_tf, self.y_inlet_tf)

        # Outlet: pressure
        # (_, _, self.p_outlet_pred) = self.net_dnn(self.x_outlet_tf, self.y_outlet_tf)
        (self.u_meu_pred) = self.net_dnn(self.x_meu_tf, self.y_meu_tf)

        # Wall: u & V
        # (self.u_wall_pred, self.v_wall_pred, _) = self.net_dnn(self.x_wall_tf, self.y_wall_tf)
        (self.u_wall_pred) = self.net_dnn(self.x_wall_tf, self.y_wall_tf)

    def load_model(self, file_dir):
        weights = []
        biases = []
        num_layers = len(self.layers)
        
        with open(file_dir, 'rb') as f:
            dnn_weights, dnn_biases = pickle.load(f)

            # stored model mush has the same layers
            assert num_layers == (len(dnn_weights)+1)

            for num in range(0, num_layers-1):
                W = tf.Variable(dnn_weights[num])
                b = tf.Variable(dnn_biases[num])
                weights.append(W)
                biases.append(b)
                print('Loaded NN parameters successfully ...')

        return weights, biases

    def initialize_adaptive_weights(self):
        # Capture gradient on each hidden layers
        self.grad_collo = []
        self.grad_boundary = []
        for lyr in range(len(self.layers)-1):
            self.grad_collo.append(tf.gradients(self.loss_collo, self.weights[lyr])[0])
            self.grad_boundary.append(tf.gradients(self.loss_bound, self.weights[lyr])[0])

        # Calculate the new weights
        self.coef_bc_list = []
        self.coef_bc_log = []
        for lyr in range(len(self.layers)-1):
            numerator_ = tf.reduce_max(tf.abs(self.grad_collo[lyr]))
            denominator_ = tf.reduce_mean(tf.abs(self.grad_boundary[lyr]))
            new_coef_bc = numerator_ / denominator_
            self.coef_bc_list.append(new_coef_bc)
        
        self.coef_bc = tf.reduce_max(tf.stack(self.coef_bc_list))

    def initialize_network(self):
        # Initialize
        weights = []
        biases = []
        num_layers = len(self.layers)

        # Create network
        for lyr in range(num_layers-1):
            # initialize weights from Xavier initialization
            np.random.seed(self.random_seed)
            W = self.xavier_init(size=[self.layers[lyr], 
                                       self.layers[lyr+1]])

            # initialize biases = 0
            np.random.seed(self.random_seed)
            b = tf.Variable(tf.zeros([1, self.layers[lyr+1]],
                                     dtype=tf.float32),
                            dtype=tf.float32)

            # Append generated weights & biases to the list
            weights.append(W)
            biases.append(b)

        return weights, biases

    def initialize_optimizers(self):
        if self.problem_type == "Inverse":
           variable_list = self.weights+self.biases+self.lambda_1
        else:
           variable_list = self.weights+self.biases
        if self.adaptive_model:
            self.global_step = tf.Variable(self.last_adam_iter, trainable=False)

            lr_init = self.lr_init
            self.lr_tf = tf.train.exponential_decay(lr_init,
                                                    self.global_step,
                                                    1000, 0.9, staircase=False)
            
            self.optimizer_adam = tf.train.AdamOptimizer(learning_rate=self.lr_tf)
            
            self.train_op_adam = self.optimizer_adam.minimize(self.loss_total,
                                                              var_list=variable_list,
                                                              global_step=self.global_step)
        else:
            self.lr_tf = tf.constant(self.lr_init, dtype=tf.float32)    
            self.optimizer_adam = tf.train.AdamOptimizer(learning_rate=self.lr_tf)
            self.train_op_adam = self.optimizer_adam.minimize(self.loss_total,
                                                            var_list=variable_list)

        self.train_op_newton = tf.contrib.opt.ScipyOptimizerInterface(
                                    self.loss_total,
                                    var_list = variable_list,
                                    method = "L-BFGS-B",
                                    options = {"maxiter": 100000,
                                               "maxfun": 100000,
                                               "maxcor": 50,
                                               "maxls": 50,
                                               "ftol": 1*np.finfo(float).eps})

    def initialize_placeholders(self):
        # Learning rate
        self.lr_tf = tf.placeholder(tf.float32, shape=[])
        self.coef_bc_tf = tf.placeholder(tf.float32, shape=self.coef_bc_val.shape)

        # Test data
        self.x_test_tf = tf.placeholder(tf.float32, shape=[None, self.x_test.shape[1]])
        self.y_test_tf = tf.placeholder(tf.float32, shape=[None, self.y_test.shape[1]])
        self.u_test_tf = tf.placeholder(tf.float32, shape=[None, self.u_test.shape[1]])
        
        # Predict data
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x_c.shape[1]])
        self.y_tf = tf.placeholder(tf.float32, shape=[None, self.y_c.shape[1]])

        # Collocation data
        self.x_c_tf = tf.placeholder(tf.float32, shape=[None, self.x_c.shape[1]])
        self.y_c_tf = tf.placeholder(tf.float32, shape=[None, self.y_c.shape[1]])

        # Boundary data
        # Inlet
#         self.x_inlet_tf = tf.placeholder(tf.float32, shape=[None, self.x_inlet.shape[1]])
#         self.y_inlet_tf = tf.placeholder(tf.float32, shape=[None, self.y_inlet.shape[1]])
#         self.u_inlet_tf = tf.placeholder(tf.float32, shape=[None, self.u_inlet.shape[1]])
        # self.v_inlet_tf = tf.placeholder(tf.float32, shape=[None, self.v_inlet.shape[1]])

        # Outlet
        self.x_meu_tf = tf.placeholder(tf.float32, shape=[None, self.x_meu.shape[1]])
        self.y_meu_tf = tf.placeholder(tf.float32, shape=[None, self.y_meu.shape[1]])
        self.u_meu_tf = tf.placeholder(tf.float32, shape=[None, self.u_meu.shape[1]])

        # Wall
        self.x_wall_tf = tf.placeholder(tf.float32, shape=[None, self.x_wall.shape[1]])
        self.y_wall_tf = tf.placeholder(tf.float32, shape=[None, self.y_wall.shape[1]])
        self.u_wall_tf = tf.placeholder(tf.float32, shape=[None, self.u_wall.shape[1]])
        # self.v_wall_tf = tf.placeholder(tf.float32, shape=[None, self.v_wall.shape[1]])

        # if self.bc_type == "Neumann":
        #    self.rho_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
        #    self.cp_tf  = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
        #    self.k_tf   = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])

    def initialize_session(self):
        tf_config = tf.ConfigProto(allow_soft_placement=True,
                                   log_device_placement=True)
        self.sess = tf.Session(config=tf_config)
        # self.sess = tf.InteractiveSession(config=tf_config)
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_variables(self):
        # For saving loss
        self.loss_total_log = []
        self.loss_collo_log = []
        self.loss_meu_log = []
        self.loss_init_log  = []
        self.loss_test_log = []
        self.count = 0
        self.last_adam_iter = 0
        self.adam_started = False
        self.newton_started = False
        self.lambda_1 = [tf.Variable([0.5], dtype=tf.float32)]
        # self.lambda_2 = tf.Variable([-6.0], dtype=tf.float32)
        self.lambda_1_log = []

    def net_dnn(self, x, y):
        # Find results
        X = tf.concat([x, y], 1)
        results = self.net_forward(X)
        
        return results
        # psi = results[:,0:1]
        # p = results[:,1:2]

        # # Find u & v from stream function (psi)
        # u = tf.gradients(psi, y)[0] / self.sigma_y
        # v = -tf.gradients(psi, x)[0] / self.sigma_x

        

    def net_forward(self, X):
        if self.improved_model:
            num_layers = len(self.weights)+1
            H = X

            encoder_1 = tf.tanh(tf.add(tf.matmul(H, self.encoder_weights_1), self.encoder_biases_1))
            encoder_2 = tf.tanh(tf.add(tf.matmul(H, self.encoder_weights_2), self.encoder_biases_2))

            for lyr in range(num_layers-2):
                W = self.weights[lyr]
                b = self.biases[lyr]
                H = tf.math.multiply(tf.tanh(tf.add(tf.matmul(H, W), b)), encoder_1) \
                    + tf.math.multiply(1 - tf.tanh(tf.add(tf.matmul(H, W), b)), encoder_2)

            W = self.weights[-1]
            b = self.biases[-1]
            Y = tf.add(tf.matmul(H, W), b)
        else:
            num_layers = len(self.weights)+1
            H = X
            # print(H)

            for lyr in range(num_layers-2):
                W = self.weights[lyr]
                b = self.biases[lyr]
                if self.act_fun == "tanh":
                    H = tf.tanh(tf.add(tf.matmul(H, W), b))
                elif self.act_fun == "relu":
                    H = tf.nn.relu(tf.add(tf.matmul(H, W), b))
                elif self.act_fun == "swish":
                    H = tf.nn.swish(tf.add(tf.matmul(H, W), b))
                elif self.act_fun == "sigmoid":
                    H = tf.nn.sigmoid(tf.add(tf.matmul(H, W), b))
                else:
                    print('No activation function')

            W = self.weights[-1]
            b = self.biases[-1]
            Y = tf.add(tf.matmul(H, W), b)

        return Y

    def net_physics(self, x, y):
        # Find results from DNN
        T = self.net_dnn(x, y)

        # Temperature gradient
        T_x = tf.gradients(T, x)[0] / self.sigma_x
        T_xx = tf.gradients(T_x, x)[0] / self.sigma_x
        T_t = tf.gradients(T, y)[0] / self.sigma_y
        # Physic Var
        rho = self.RHO
        cp  = self.cp
        k   = self.k
        M   = self.M

        # Physics error
        if self.problem_type == "Inverse":
           lambda_1 = self.lambda_1
           x = self.x_c
           y = self.y_c
           # x = (self.x_c - self.mu_x) / self.sigma_x
           # y = (self.y_c - self.mu_y) / self.sigma_y
           f = lambda_1*T_t - T_xx - np.exp(x + 2*y)
        else:
           if self.bc_type == 'Dirichlet':
              x = self.x_c
              y = self.y_c
              # x = (self.x_c - self.mu_x) / self.sigma_x
              # y = (self.y_c - self.mu_y) / self.sigma_y
              f = T_t - T_xx - np.exp(x + 2*y)
           elif self.bc_type == 'Neumann':
              # f = rho*cp*T_t - k*T_xx
              k_ = (T - k[0,0])/(k[1,0] - k[0,0])*(k[1,1] - k[0,1]) + k[0,1]
              rho_ = (T - rho[0,0])/(rho[1,0] - rho[0,0])*(rho[1,1] - rho[0,1]) + rho[0,1]
              cp_ = (T - cp[0,0])/(cp[1,0] - cp[0,0])*(cp[1,1] - cp[0,1]) + cp[0,1]
              f = M*T_t - M*T_xx*k_/(rho_*cp_)

        return f
    def normalize_data(self, data, axis):
        if axis == "x":
            normalized_data = (data - self.mu_x) / self.sigma_x
            # normalized_data = data
            # normalized_data = data / self.ub[0]
        elif axis == "y":
            normalized_data = (data - self.mu_y) / self.sigma_y
            # normalized_data = data
            # normalized_data = data / self.ub[1]

        return normalized_data

    def predict(self, x_star, y_star):
        # Prepare the input
        x_star = (x_star - self.mu_x) / self.sigma_x
        y_star = (y_star - self.mu_y) / self.sigma_y

        # x_star = x_star / self.ub[0]
        # y_star = y_star / self.ub[1]

        # Create dictionary
        tf_dict = {self.x_tf:x_star, self.y_tf:y_star}

        # Predict
        u_star = self.sess.run(self.u_pred, tf_dict)
        # v_star = self.sess.run(self.v_pred, tf_dict)
        # p_star = self.sess.run(self.p_pred, tf_dict)

        return u_star

    def save_loss(self, file_dir):
        loss_test = np.array(self.loss_test_log)
        loss_data = np.column_stack((self.loss_total_log, 
                                     self.loss_collo_log,
                                     self.loss_bound_log,
                                     self.loss_init_log,
                                     loss_test))
        loss_df = pd.DataFrame(loss_data, columns=["total", "collo", "boundary", "initial", "error_u"])
        joblib.dump(loss_df, file_dir)

    def save_model(self, file_dir):
        weights = self.sess.run(self.weights)
        biases = self.sess.run(self.biases)

        with open(file_dir, 'wb') as f:
            pickle.dump([weights, biases], f)
            print("Save NN parameters successfully...")

    def test_loss(self):
        # Prediction
        # u_pred, v_pred, p_pred = self.predict(self.x_test, self.y_test)
        u_pred = self.predict(self.x_test, self.y_test)
        error_u = np.linalg.norm(u_pred - self.u_test) / np.linalg.norm(self.u_test)

        return error_u

    def unpack(self, data, params):
        # Initialize
        self.data = data
        self.params = params

        # Unpack Parameters
        # Data-Boundary
        self.lb = params["data"]["lb"]
        self.ub = params["data"]["ub"]
        
        self.random_seed = data["train"]["random_seed"]
        
        # Data-Collocation
        self.x_c = data["train"]["collo"][:, 0:1]
        self.y_c = data["train"]["collo"][:, 1:2]
        self.mu_x = data["train"]["mu_x"]
        self.mu_y = data["train"]["mu_y"]
        self.sigma_x = data["train"]["sigma_x"]
        self.sigma_y = data["train"]["sigma_y"]
        
        # Measurements
        self.loc_num = params["data"]["loc"]
        self.meu_num = params["data"]["meu"]
#         meu = np.zeros(shape=(1, 3))
#         for i in range(0, len(self.loc_num) ):
#             meu = np.concatenate((meu, self.data["train"][f"loc={self.loc_num[i]}"]), axis=0)
            
#         meu = meu[1:,:]
        meu = data["train"]["mea"]
        self.x_meu = meu[:, 0:1]
        self.y_meu = meu[:, 1:2]
        self.u_meu = meu[:, 2:3]
        
        # Data-Inlet
#         self.x_inlet = data["train"]["inlet"][:, 0:1]
#         self.y_inlet = data["train"]["inlet"][:, 1:2]
#         self.u_inlet = data["train"]["inlet"][:, 2:3]
        # self.v_inlet = data["train"]["inlet"][:, 3:4]
        # self.p_inlet = data["train"]["inlet"][:, 4:5]

        # Data-Outlet
#         self.x_outlet = data["train"]["outlet"][:, 0:1]
#         self.y_outlet = data["train"]["outlet"][:, 1:2]
#         self.u_outlet = data["train"]["outlet"][:, 2:3]
        # self.v_outlet = data["train"]["outlet"][:, 3:4]
        # self.p_outlet = data["train"]["outlet"][:, 4:5]
        
        # Data-Wall
        self.x_wall = data["train"]["wall"][:, 0:1]
        self.y_wall = data["train"]["wall"][:, 1:2]
        self.u_wall = data["train"]["wall"][:, 2:3]
        # self.v_wall = data["train"]["wall"][:, 3:4]
        # self.p_wall = data["train"]["wall"][:, 4:5]

        # Data-Test
        self.x_test = data["test"][:, 0:1]
        self.y_test = data["test"][:, 1:2]
        self.u_test = data["test"][:, 2:3]
        # self.v_test = data["test"][:, 3:4]
        # self.p_test = data["test"][:, 4:5]

        # Data-Others
        self.n_collo = params["data"]["n_collo"]
        self.M = params["data"]["M"]
        self.x = np.linspace(0,0.1,101).flatten()[:,None]
        
        # # Physic
        self.RHO = params["physic"]["rho"]
        self.cp  = params["physic"]["cp"]
        self.k   = params["physic"]["k"]
        # self.MU = params["physic"]["mu"]

        # Network
        self.epochs = params["network"]["epochs"]
        self.layers = params["network"]["layers"]
        self.coef_bc_val = np.array(params["network"]["coef_bc"])
        self.lr_init = params["network"]["lr_init"]
        self.beta = params["network"]["beta"]
        self.batch_size = params["network"]["batches"]
        self.verboses_adam = params["network"]["verboses_adam"]
        self.verboses_newton = params["network"]["verboses_newton"]
        self.verboses_adaptive = params["network"]["verboses_adaptive"]
        self.saver = params["network"]["saver"]
        self.act_fun = params["network"]["act_fun"]

    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2 / (in_dim + out_dim))

        # tf.set_random_seed(20)
        np.random.seed(self.random_seed)
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], 
                                               stddev=xavier_stddev, 
                                               dtype=tf.float32,
                                               seed=self.random_seed), 
                           dtype=tf.float32)


### **Cases**

In [ ]:
class CasesChannel:

    def __init__(self, add_noise=False, save_fig=False):
        self.add_noise = add_noise
        self.save_fig = save_fig
        self.data = {}

    # def analytics_solution(self, y):
    def analytics_solution(self, x, t):
        T_ = np.exp(x+2*t)

        return T_

    def generate_bc(self, loc, n):
        x = loc*self.ub[0]*np.ones((n,1))
        np.random.seed(self.random_seed)  #INI BEDA
        t = lhs(1,n)*self.ub[1]
        T = self.analytics_solution(x, t)
        # print(T)

        # concat data
        bc_datas = np.concatenate((x, t, T), axis=1)
        
        return bc_datas

    def generate_ic(self):
        n = self.n_wall
        np.random.seed(self.random_seed)      #INI BEDA
        x = lhs(1,n)*self.ub[0]
        t = 0.0*lhs(1,n)
        T = self.analytics_solution(x, t)

        # concat data
        ic_datas = np.concatenate((x, t, T), axis=1)
#         print(ic_datas[:,0].shape)
        return ic_datas #, random_noise
        
    
    def generate_collo(self):
        # unpack
        n = self.n_collo
        lb = self.lb
        ub = self.ub

        # create points
        np.random.seed(self.random_seed)
        collo_pts = lb + (ub-lb)*lhs(2, n, criterion=None)

        self.mu_X, self.sigma_X = collo_pts.mean(0), collo_pts.std(0)
        self.mu_x, self.sigma_x = self.mu_X[0], self.sigma_X[0]
        self.mu_y, self.sigma_y = self.mu_X[1], self.sigma_X[1]
                
        return collo_pts 

    def generate_train_data(self, param, bc_type):
        self.bc_type = bc_type
        
        # Unpack parameters
        self.unpack(param)
        

        # Generate points
        # Generate boundary conditions data 
        init_cond = self.generate_ic()
        
        loc = self.loc_num   # location measurements --> e.g 0%, 50%, 100%
        meu = self.meu_num   # Measurements --> 50, 100, 150..
        
        bc = []
#         noise_bc = []
        
        self.data["train"] = {}
        for i in range (0, len(loc)):
            mea = self.generate_bc(loc[i],int(meu[i]))
            bc.append(mea)
#             noise_bc.append(mea_noise)
            self.data["train"][f"loc={loc[i]}"] = bc[i]
#             self.data["train"][f"noise_loc={loc[i]}"] = noise_bc[i]
            
        mea = np.zeros(shape=(1, 3))
        for i in range(0, len(self.loc_num) ):
            mea = np.concatenate((mea, self.data["train"][f"loc={self.loc_num[i]}"]), axis=0)
            
        mea = mea[1:,:]
#         self.x_meu = meu[:, 0:1]
#         self.y_meu = meu[:, 1:2]
        T = mea[:, 2:3]
        T_init = init_cond[:,2:3]
        self.T = T
        self.T_init= T_init
            
        # noise addition
        if self.add_noise:
            noise = self.noise_level
            maxT = max([np.max(T), np.max(T_init)])
            sigma = noise * maxT
            print(f"T max: {maxT}")
            np.random.seed(self.random_seed)
            random_noise_T = sigma*np.random.randn(T.shape[0],1)
            
            T = T + random_noise_T
            
            random_noise_init = sigma*np.random.randn(T_init.shape[0], 1)
            
            T_init = T_init + random_noise_init
            
        self.random_noise_T = random_noise_T
        self.random_noise_init = random_noise_init
            
        self.data["train"]["mea"] = np.concatenate((mea[:, 0:2], T), axis=1)         
        self.data["train"]["wall"] = np.concatenate((init_cond[:, 0:2], T_init), axis=1)
            

        # Generate collocations points data
        collo_ = self.generate_collo()

        # Pack data
#         self.data["train"] = {}
        self.data["train"]["collo"] = collo_
#         self.data["train"]["inlet"] = left_bc
#         self.data["train"]["meu"] = right_bc
#          = init_cond
        self.data["train"]["mu_x"] = self.mu_x
        self.data["train"]["sigma_x"] = self.sigma_x
        self.data["train"]["mu_y"] = self.mu_y
        self.data["train"]["sigma_y"] = self.sigma_y
        self.data["train"]["random_seed"] = self.random_seed

    def generate_test_data(self, param):
        # Unpack parameters
        self.unpack(param)

        # Generate points
        length_ = self.ub - self.lb
        min_length_ = np.argmin(length_)
        frac_ = max(length_) / min(length_)
        n_test_1 = int(frac_ * self.n_test)

        # Create grid
        if min_length_ == 0:
            self.n_x = self.n_test
            self.n_y = n_test_1
        else:
            self.n_x = n_test_1
            self.n_y = self.n_test
        if self.bc_type == "Dirichlet":
            x_ = np.linspace(self.lb[0], self.ub[0], num=(self.n_x)) # tadinya n_test
            # x_add = np.column_stack((x_1, x_2))
            y_ = np.linspace(self.lb[1], self.ub[1], num=int(self.n_y/self.n_test))
            X, Y = np.meshgrid(x_, y_)
            X_flat = X.flatten()[:,None]
            Y_flat = Y.flatten()[:,None]
            T_ = self.analytics_solution(X_flat, Y_flat)
        elif self.bc_type == "Neumann":
            u_fea = pd.read_csv('import_test_t2160.csv')
            u_fea = u_fea.to_numpy()
            # x_ = np.linspace(self.lb[0], self.ub[0], num=self.n_x)
            x_ = u_fea[:,0:1]
            y_ = self.t_check
            X, Y = np.meshgrid(x_, y_)
            X_flat = X.flatten()[:,None]
            Y_flat = Y.flatten()[:,None]
            T_ = u_fea[:,1:2]
            # T_ = u_fea[:,1:2] + 273.0
        # T_1 = self.analytics_solution(x_1,y_)
        # T_2 = self.analytics_solution(x_2,y_)
        # T_add = np.column_stack((T_1, T_2))
        # v_ = np.zeros(self.n_test)

        # self.data["test"] = np.column_stack((x_, y_, u_, v_))
        self.data["test"] = np.column_stack((X_flat, Y_flat, T_))
        # self.data["test_add"] = np.column_stack((x_1, x_2, y_, T_1, T_2))

    def plot(self):
        # unpack data
        collo_ = self.data["train"]["collo"]
        meu = []
        
        for i in range(0, len(self.loc_num)):
#             print(self.data["train"])
#                        self.data["train"][f"loc={loc[i]}"]
            meu.append(self.data["train"][f"loc={self.loc_num[i]}"])
#         inlet_ = self.data["train"]["inlet"]
#         outlet_ = self.data["train"]["outlet"]
        wall_ = self.data["train"]["wall"]

        # PLOT: points distribution
        # Properties
        plt.rc('text', usetex=True)
        plt.rc('font', family='serif')

        # Plot
        if self.bc_type == "Dirichlet":
            fig_w = 5
            fig_h = 5
        elif self.bc_type == "Neumann":
            fig_w = 3
            fig_h = 10
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h), constrained_layout=True, dpi=300)

        ax.plot([self.lb[0], self.ub[0]], [self.lb[1], self.lb[1]], 'k')
        ax.plot([self.lb[0], self.ub[0]], [self.ub[1], self.ub[1]], 'k')
        ax.plot([self.lb[0], self.lb[0]], [self.ub[1], self.lb[1]], 'k')
        ax.plot([self.ub[0], self.ub[0]], [self.ub[1], self.lb[1]], 'k')

        ax.scatter(collo_[:,0:1], collo_[:,1:2], marker='.', alpha=0.7, c='grey', label='Collo')
        for i in range(0, len(self.loc_num)):
            ax.scatter(meu[i][:,0:1], meu[i][:,1:2], marker='.', alpha=0.7,  label=f'Measurements, $x/L$={np.round(self.loc_num[i],2)}')
#         ax.scatter(inlet_[:,0:1], inlet_[:,1:2], marker='.', alpha=0.7, c='r', label='Left BC')
#         ax.scatter(outlet_[:,0:1], outlet_[:,1:2], marker='.', alpha=0.7, c='g', label='Right BC')
        ax.scatter(wall_[:,0:1], wall_[:,1:2], marker='.', alpha=0.7, c='b', label='IC')

        ax.set_title("Points distribution", fontsize=15)
        ax.set_xlabel("$x$ (m)", fontsize=20)
        ax.set_ylabel("$t$ (s)", fontsize=20)
#         ax.set_xlabel("x (m)", fontsize=12)
#         ax.set_ylabel("t (s)", fontsize=12)
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.legend(fontsize=10, loc=4)
        ax.grid(linestyle="--")
        ax.set_xlim(self.lb[0], self.ub[0])
        ax.set_ylim(self.lb[1], self.ub[1])

        if self.save_fig:
            fig.savefig("fig_point_distribution.eps", format="eps")
        plt.show()

        # # PLOT: test data
        # fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 3), constrained_layout=True, dpi=300)

        # # Inlet velocity
        # x_ = 0.5*(self.ub[0]-self.lb[0])*np.ones(self.n_test)
        # y_ = np.linspace(self.lb[1], self.ub[1], self.n_inlet)
        # u_analytics = self.analytics_solution(x_,y_)

        # ax.plot(u_analytics, y_, 'b', label='fully-developed flow')
        # ax.scatter(inlet_[:, 2:3], inlet_[:, 1:2], c='r', marker='.', label='inlet')

        # ax.set_title("Velocity Comparison", fontsize=20)
        # #ax.set_xlabel('$u$ (m/s)', fontsize=20)
        # #ax.set_ylabel('$y$ (m)', fontsize=20)
        # ax.set_xlabel('u (m/s)', fontsize=20)
        # ax.set_ylabel('y (m)', fontsize=20)
        # ax.tick_params(axis='both', which='major', labelsize=16)
        # ax.legend(fontsize=12)
        # ax.grid(linestyle="--")
        # ax.set_ylim([self.lb[1], self.ub[1]])

        if self.save_fig:
            fig.savefig("fig_references.eps", format="eps")
        plt.show()

    def unpack(self, param):
        # # Constant
        # self.mu = param["physic"]["mu"]
        # self.u_max = param["physic"]["u_max"]
        # self.u_wall = param["physic"]["u_wall"]

        # self.T_left = param["physic"]["T_left"]
        # self.T_right = param["physic"]["T_right"]
        # self.T_lower = param["physic"]["T_lower"]
        # self.T_upper = param["physic"]["T_upper"]

        # Bound
        self.lb = param["data"]["lb"]
        self.ub = param["data"]["ub"]

        # Discretizations
        self.n_collo = param["data"]["n_collo"]
#         self.n_inlet = param["data"]["n_inlet"]
#         self.n_outlet = param["data"]["n_outlet"]
        self.n_wall = param["data"]["n_wall"]
        self.n_test = param["data"]["n_test"]
        self.init_wall = param["data"]["init_wall"]
        self.left_q = param["data"]["left_q"]
        self.right_q = param["data"]["right_q"]
        self.t_check = param["data"]["t_check"]
        self.random_seed = param["data"]["seed"]
        self.noise_level = param["data"]["noise"]
        self.loc_num = param["data"]["loc"]
        self.meu_num = param["data"]["meu"]


### **Post Processing**

In [ ]:
class PostChannel:

    def __init__(self, model, params, save_fig=False):
        self.save_fig = save_fig
        self.unpack(model, params)

    def calculate_pinn(self):
        
        # self.u_pinn, self.v_pinn, self.p_pinn = self.model.predict(self.X_flat, self.Y_flat)
        # self.u_pinn = self.model.predict(self.X_flat, self.Y_flat/self.ub[1])
        if self.model.bc_type == "Neumann":
            n_y = self.n_y_dummy
        elif self.model.bc_type == "Dirichlet":
            n_y = self.n_y
        self.u_pinn = self.model.predict(self.X_flat, self.Y_flat)
        self.x_pinn_ = self.X_flat.reshape(n_y, self.n_x)
        self.y_pinn_ = self.Y_flat.reshape(n_y, self.n_x)
        self.u_pinn_ = self.u_pinn.reshape(n_y, self.n_x)
        # self.v_pinn_ = self.v_pinn.reshape(self.n_y, self.n_x)
        # self.p_pinn_ = self.p_pinn.reshape(self.n_y, self.n_x)

    def calculate_error(self, n_data):
        # X & Y
        x_test = self.model.x_test # x = 0.25
        y_test = self.model.y_test

        # Predict
        # u_pinn, _, _ = self.model.predict(x_test, y_test)
        # u_pinn = self.model.predict(x_test, y_test)
        
        # # Find error
        # del_u = u_pinn - u_test
        # rmse_u = np.linalg.norm(del_u) / np.linalg.norm(u_test)


        # Prediction
        u_pinn = self.u_pinn

        if self.model.bc_type == 'Dirichlet':
            # u_analytic = case.analytics_solution(self.X_flat, self.Y_flat)
            u_analytic = np.exp(self.X_flat+2*self.Y_flat)
            u_test = u_analytic
            self.u_test = u_test
            # u_test = self.model.u_test
        elif self.model.bc_type == 'Neumann':
            u_fea = pd.read_csv('import_test_t2160.csv')
            u_test = u_fea

        delta_u = np.abs(u_pinn - u_test)
        #  Absolute error with n_x and n_y points
        self.abs_err_u = np.sum(delta_u)/(self.n_x * self.n_y)

        rel_err_u_ij = delta_u/u_test
        # Relative error with n_x and n_y points
        self.rel_err_u = np.sum(rel_err_u_ij)/(self.n_x * self.n_y)
        lambda_ = self.model.lambda_1_log[-1][0][0]
#         lambda_2 = self.model.lambda_2_log[-1][0][0]
        
        self.rel_err_lambda = np.abs(lambda_ - 1.0) / 1.0
#         self.rel_err_lambda_2 = np.abs(lambda_2 - 1.0) / 1.0
        

        # rmse_total = rmse_u
        # print(f"- RMSE x 10^(-2): {rmse_total/(1.e-2):5f}")
        
        print(f"- Relative Error Lambda (%): {self.rel_err_lambda*100:5f}")

        # rmse_total = rmse_u
        # print(f"- RMSE x 10^(-2): {rmse_total/(1.e-2):5f}")
        print(f"- Absolute Error: {self.abs_err_u:5f}")
        print(f"- Relative Error (%): {self.rel_err_u*100:5f}")
        

        return x_test, y_test, u_pinn, u_test

    def create_test_data(self):
        # Find fraction
        length_ = self.ub - self.lb
        min_length_ = np.argmin(length_)
        frac_ = max(length_) / min(length_)
        n_test_1 = int(frac_ * self.n_test)

        # Create grid
        if min_length_ == 0:
            self.n_x = self.n_test
            self.n_y = n_test_1
        else:
            self.n_x = n_test_1
            self.n_y = self.n_test

        if self.model.bc_type == 'Dirichlet':
            self.x = np.linspace(self.lb[0], self.ub[0], num=self.n_x)
            self.y = np.linspace(self.lb[1], self.ub[1], num=self.n_y)
            self.X, self.Y = np.meshgrid(self.x, self.y)
            self.X_flat = self.X.flatten()[:, None]
            self.Y_flat = self.Y.flatten()[:, None]
        elif self.model.bc_type == 'Neumann':
            self.x = np.linspace(self.lb[0], self.ub[0], num=self.n_x)
            # self.y = 2160*np.ones(self.n_x)
            self.n_y_dummy = int(self.n_y / self.n_test)
            self.y = np.linspace(self.lb[1], self.ub[1], num=self.n_y_dummy)
            self.X, self.Y = np.meshgrid(self.x, self.y)
            self.X_flat = self.X.flatten()[:,None]
            self.Y_flat = self.Y.flatten()[:,None]

    def display_loss(self):
        # SET
        #plt.rc('text', usetex=True)
        #plt.rc('font', family='serif')

        # Extract data
        loss_total = np.sqrt(self.model.loss_total_log)
        loss_collo = np.sqrt(self.model.loss_collo_log)
        loss_meu = np.sqrt(self.model.loss_meu_log)
        loss_init = np.sqrt(self.model.loss_init_log)
        loss_test = self.model.loss_test_log

        # Status
        if self.model.adam_started==True and self.model.newton_started==True:
            run_status = "full"
        elif self.model.adam_started:
            run_status = "adam"
        elif self.model.newton_started:
            run_status = "newton"
        else:
            run_status = "none"

        # Prepare
        iter_total = []

        if run_status == "full":
            n_total = len(loss_total)
            n_adam = int(self.model.last_adam_iter / self.model.verboses_adam)
            n_newton = n_total - n_adam

            for i in range(n_total):
                if i < n_adam:
                    iter_total.append(i * self.verboses_adam)
                else:
                    iter_total.append(self.model.last_adam_iter + i)
        elif run_status == "adam":
            n_total = int(self.model.last_adam_iter // self.model.verboses_adam)

            if self.model.last_adam_iter % self.model.verboses_adam != 0:
                n_total += self.model.last_adam_iter % self.model.verboses_adam

            for i in range(n_total):
                iter_total.append(i * self.verboses_adam)
        elif run_status == "newton":
            n_total = len(loss_total)

            for i in range(n_total):
                iter_total.append(i)
        else:
            n_total = 0
        
        # PLOT History
        if run_status != "none":
            fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6), constrained_layout=True, dpi=300)
            ax.plot(iter_total, loss_total, "r", linestyle="solid", label="Physical Loss")
            ax.plot(iter_total, loss_collo, "g", linestyle="dashdot", label="Collocation Loss")
            ax.plot(iter_total, loss_meu, "b", linestyle="dotted", label="Measurement Loss")
            ax.plot(iter_total, loss_init, 'purple',linestyle='solid', label='Initial Loss')
            ax.plot(iter_total, loss_test, "black", linestyle="dashed", label="Test Error")

            if run_status == "full":
                min_ = min(min(loss_total, loss_collo, loss_bound, loss_test))
                min_ = min_ - 10*min_
                max_ = max(max(loss_total, loss_collo, loss_bound, loss_test))
                max_ = max_ + 10*max_

                x_ = self.model.last_adam_iter
                ax.plot([x_, x_], [min_, max_], "cyan", linestyle="dashed")

            #ax.set_title("Loss History", fontsize=20)
            ax.set_xlim(0, iter_total[-1])
            ax.set_yscale("log")
            ax.set_xlabel("Iterations", fontsize=12)
            ax.set_ylabel("RMSE", fontsize=12)
            ax.grid(linestyle="--")
            ax.legend(fontsize=10)
            ax.set_title('Loss History (Log Scale)', fontsize=17)
            ax.tick_params(axis='both', which='major', labelsize=10)

            # save figure
            if self.save_fig:
                fig.savefig("fig_loss_history.eps", format="eps")
            plt.show()

        # PLOT Weights history
        if self.model.adaptive_model:
            adaptive_weights = self.model.coef_bc_log
            iter_weights = [i*self.verboses_adaptive for i in range(len(adaptive_weights))]

            # Create figure
            fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 4), constrained_layout=False, dpi=300)
            
            ax.plot(iter_weights, adaptive_weights, "r", linestyle="solid", label="Boundary Weights")
            ax.set_title("Boundary Weights", fontsize=10)
            ax.set_xlim(min(iter_weights), max(iter_weights))
            ax.set_xlabel("Iterations", fontsize=10)
            ax.set_ylabel("Boundary Weights", fontsize=10)
            ax.grid(linestyle="--")
            ax.tick_params(axis='both', which='major', labelsize=14)
            ax.legend(fontsize=6)
            
            # save figure
            if self.save_fig:
                fig.savefig("fig_weights_history.eps", format="eps")
            plt.show()
        
        print(f"- Last Iterations: {iter_total[-1]}")
        
    def display_contour(self):
        # FIND: PINN results
        self.create_test_data()
        self.calculate_pinn()

        # PLOT CONTOUR
        fig_h = 3 
        fig_w = 10

        # CALCULATE: error
        if self.model.bc_type == "Dirichlet":
            x_, y_, u_pinn, u_test = self.calculate_error(n_data = self.n_test)
            # PLOT: comparison
            self.plot_comparison(x=self.x, y=self.y, 
                                phi_pinn=u_pinn.reshape(self.n_y, self.n_x), 
                                phi_analytics=u_test.reshape(self.n_y, self.n_x), 
                                fig_w=fig_w, fig_h=fig_h*2)
            self.plot_comparison2(x=self.x, y=self.y, 
                                 phi_pinn=u_pinn.reshape(self.n_y, self.n_x), 
                                 phi_analytics=u_test.reshape(self.n_y, self.n_x), 
                                 fig_w=fig_w, fig_h=fig_h*2)
            v_max = max(u_test)
            v_min = 0.0
        elif self.model.bc_type == "Neumann":
            self.neumann_loss()
            v_max = int(max(self.u_pinn))
            v_min = case.init_wall
            fig_w = int(fig_w*2/3)

        # PLOT: u_velocity
        
        self.plot_countour(x=self.X, y=self.Y, 
                           x_flat=self.X_flat, y_flat=self.Y_flat,
                           phi=self.u_pinn, phi_=self.u_pinn_, 
                           vmin=v_min, vmax=v_max,
                           types="T", 
                           fig_w=fig_w, fig_h=fig_h)

        # # PLOT: v_velocity
        # self.plot_countour(x=self.X, y=self.Y, 
        #                    x_flat=self.X_flat, y_flat=self.Y_flat,
        #                    phi=self.v_pinn, phi_=self.v_pinn_, 
        #                    vmin=-0.3, vmax=0.3,
        #                    types="v", 
        #                    fig_w=fig_w, fig_h=fig_h)
        
        # # PLOT: p_pressure
        # self.plot_countour(x=self.X, y=self.Y, 
        #                    x_flat=self.X_flat, y_flat=self.Y_flat,
        #                    phi=self.p_pinn, phi_=self.p_pinn_, 
        #                    vmin=0.0, vmax=5.0,
        #                    types="p", 
        #                    fig_w=fig_w, fig_h=fig_h)

    def plot_comparison(self, x, y, phi_pinn, phi_analytics, fig_w, fig_h):
        #plt.rc('text', usetex=True)
        #plt.rc('font', family='serif')
        # phi_pinn = phi_pinn.
        # phi_analytics = phi_analytics.reshape(self.n_y, self.n_x)
        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(fig_w, fig_h), dpi=100, constrained_layout=False)
        
        # print(phi_pinn)
        ax[1].plot(y, phi_pinn[:,self.n_x//2].flatten(), 'b', label="PINN solution", linewidth=4)
        ax[1].plot(y, phi_analytics[:,self.n_x//2], '--r', label="Analytical solution", linewidth=4)
        

        x_lim_min = min(np.min(phi_pinn), np.min(phi_analytics))
        x_lim_max = max(np.max(phi_pinn), np.max(phi_analytics))
        x_lim_min = 0
        x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
        ax[1].set_ylim(x_lim_min, x_lim_max)
        ax[1].set_xlim(self.lb[1], self.ub[1])
        #ax.set_xlabel("$u$ (m/s)", fontsize=20)
        #ax.set_ylabel("$y$ (m)", fontsize=20)
        ax[1].set_xlabel("$t$ (s), $x=0.5$ m", fontsize=15)
        ax[1].set_ylabel("$T$ (\u2103)", fontsize=15)
        ax[1].grid(linestyle="--")
        ax[1].legend(fontsize=10)
        #ax.set_title("$u$-velocity comparison", fontsize=20)
        ax[1].set_title("Temperature comparison", fontsize=20)
        ax[1].tick_params(axis='both', which='major', labelsize=16)

        ax[0].plot(y, phi_pinn[:,self.n_x//4].flatten(), 'b', label="PINN solution", linewidth=4)
        ax[0].plot(y, phi_analytics[:,self.n_x//4], '--r', label="Analytical solution", linewidth=4)
        
        # x_lim_min = min(min(phi_pinn[:,1:2].flatten()), min(phi_analytics[:,1:2]))
        # x_lim_max = max(max(phi_pinn[:,1:2].flatten()), max(phi_analytics[:,1:2]))
        # x_lim_min = x_lim_min - 0.2*abs(x_lim_min)
        # x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
        ax[0].set_ylim(x_lim_min, x_lim_max)
        ax[0].set_xlim(self.lb[1], self.ub[1])
        #ax.set_xlabel("$u$ (m/s)", fontsize=20)
        #ax.set_ylabel("$y$ (m)", fontsize=20)
        ax[0].set_xlabel("$t$ (s), $x=0.25$ m", fontsize=15)
        ax[0].set_ylabel("$T$ (\u2103)", fontsize=15)
        ax[0].grid(linestyle="--")
        ax[0].legend(fontsize=10)
        #ax.set_title("$u$-velocity comparison", fontsize=20)
        # ax[0].set_title("Temperature comparison", fontsize=20)
        ax[0].tick_params(axis='both', which='major', labelsize=16)

        ax[2].plot(y, phi_pinn[:,self.n_x//4*3+1].flatten(), 'b', label="PINN solution", linewidth=4)
        ax[2].plot(y, phi_analytics[:,self.n_x//4*3+1], '--r', label="Analytical solution", linewidth=4)
        
        # x_lim_min = min(min(phi_pinn[:,2:3].flatten()), min(phi_analytics[:,2:3]))
        # x_lim_max = max(max(phi_pinn[:,2:3].flatten()), max(phi_analytics[:,2:3]))
        # x_lim_min = x_lim_min - 0.2*abs(x_lim_min)
        # x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
        ax[2].set_ylim(x_lim_min, x_lim_max)
        ax[2].set_xlim(self.lb[1], self.ub[1])
        #ax.set_xlabel("$u$ (m/s)", fontsize=20)
        #ax.set_ylabel("$y$ (m)", fontsize=20)
        ax[2].set_xlabel("$t$ (s), $x=0.75$ m", fontsize=15)
        ax[2].set_ylabel("$T$ (\u2103)", fontsize=15)
        ax[2].grid(linestyle="--")
        ax[2].legend(fontsize=10)
        #ax.set_title("$u$-velocity comparison", fontsize=20)
        # ax[2].set_title("Temperature comparison", fontsize=20)
        ax[2].tick_params(axis='both', which='major', labelsize=16)
        plt.tight_layout()

    def plot_comparison2(self, x, y, phi_pinn, phi_analytics, fig_w, fig_h):
        #plt.rc('text', usetex=True)
        #plt.rc('font', family='serif')
        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(fig_w, fig_h), dpi=100, constrained_layout=False)
        
        # print(phi_pinn)
        ax[1].plot(x, phi_pinn[self.n_x//2,:].flatten(), 'b', label="PINN solution", linewidth=4)
        ax[1].plot(x, phi_analytics[self.n_x//2,:], '--r', label="Analytical solution", linewidth=4)
        

        x_lim_min = min(np.min(phi_pinn), np.min(phi_analytics))
        x_lim_max = max(np.max(phi_pinn), np.max(phi_analytics))
        x_lim_min = 0
        x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
        ax[1].set_ylim(x_lim_min, x_lim_max)
        ax[1].set_xlim(self.lb[1], self.ub[1])
        ax[1].set_xlabel("$x$ (m), $t=0.5$ s ", fontsize=18)
        ax[1].set_ylabel("$T$ (\u2103)", fontsize=18)
        ax[1].grid(linestyle="--")
        ax[1].legend(fontsize=10)
        ax[1].set_title("Temperature comparison", fontsize=25)
        ax[1].tick_params(axis='both', which='major', labelsize=19)

        ax[0].plot(x, phi_pinn[self.n_x//4,:].flatten(), 'b', label="PINN solution", linewidth=4)
        ax[0].plot(x, phi_analytics[self.n_x//4,:], '--r', label="Analytical solution", linewidth=4)
        
        ax[0].set_ylim(x_lim_min, x_lim_max)
        ax[0].set_xlim(self.lb[1], self.ub[1])
        ax[0].set_xlabel("$x$ (m), $t=0.25$ s ", fontsize=18)
        ax[0].set_ylabel("$T$ (\u2103)", fontsize=18)
        ax[0].grid(linestyle="--")
        ax[0].legend(fontsize=10)
        ax[0].tick_params(axis='both', which='major', labelsize=19)

        ax[2].plot(x, phi_pinn[self.n_x//4*3+1,:].flatten(), 'b', label="PINN solution", linewidth=4)
        ax[2].plot(x, phi_analytics[self.n_x//4*3+1,:], '--r', label="Analytical solution", linewidth=4)
        
        ax[2].set_ylim(x_lim_min, x_lim_max)
        ax[2].set_xlim(self.lb[1], self.ub[1])
        ax[2].set_xlabel("$x$ (m), $t=0.75$ s ", fontsize=18)
        ax[2].set_ylabel("$T$ (\u2103)", fontsize=18)
        ax[2].grid(linestyle="--")
        ax[2].legend(fontsize=10)
        ax[2].tick_params(axis='both', which='major', labelsize=19)
        plt.tight_layout()
        
    def plot_countour(self, x, y, x_flat, y_flat, phi, phi_, vmin, vmax, types, fig_w, fig_h):
        # Set
        #plt.rc('text', usetex=True)
        #plt.rc('font', family='serif')
        
        # Create plot
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h), dpi=300, constrained_layout=False)

        # Unpack
        cf = ax.scatter(x_flat, y_flat, c=phi, 
                        alpha=1., edgecolors='none', cmap='jet', marker=".", s=50, vmin=vmin, vmax=vmax)
        ax.set_xlim(self.lb[0], self.ub[0])
        ax.set_ylim(self.lb[1], self.ub[1])
        
        if types=="p":
            #ax.set_title(f'${types}$ (Pa)', fontsize=20)
            ax.set_title(f'{types} (Pa)', fontsize=20)
        else:
            #ax.set_title(f'${types}$ (m/s)', fontsize=20)
            ax.set_title(f'${types} (x,t)$ PINN Solution', fontsize=20)
        
        #ax.set_xlabel('$x$ (m)', fontsize=20)
        #ax.set_ylabel('$y$ (m)', fontsize=20)
        ax.set_xlabel('$x$ (m)', fontsize=20)
        ax.set_ylabel('$t$ (s)', fontsize=20)
        ax.tick_params(axis='both', which='major', labelsize=16)
        ax.contour(x, y, phi_, colors='k', linewidths=0.2, levels=50)
        
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="2%", pad=0.1)
        cb = fig.colorbar(cf, cax=cax)
        cb.ax.set_title('$T$(\u2103)', fontsize=18)
        ticks = np.linspace(vmin, vmax, 3)
        ticks = np.floor(ticks)
        cb.set_ticks(ticks)
        cb.ax.tick_params(labelsize=16)

        if self.model.bc_type == 'Dirichlet':
          # Analytical Solution contour
          T = np.exp(x_flat+2*y_flat)
          fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h), dpi=300, constrained_layout=False)
          cf = ax.scatter(x_flat, y_flat, c=T, alpha=1., edgecolors='none', cmap='jet', marker=".", s=50, vmin=vmin, vmax=vmax)
          ax.set_xlim(self.lb[0], self.ub[0])
          ax.set_ylim(self.lb[1], self.ub[1])
          
          if types=="p":
              #ax.set_title(f'${types}$ (Pa)', fontsize=20)
              ax.set_title(f'{types} (Pa)', fontsize=20)
          else:
              #ax.set_title(f'${types}$ (m/s)', fontsize=20)
              ax.set_title(f'${types} (x,t)$ Analytical Solution', fontsize=20)
          
          #ax.set_xlabel('$x$ (m)', fontsize=20)
          #ax.set_ylabel('$y$ (m)', fontsize=20)
          ax.set_xlabel('$x$ (m)', fontsize=20)
          ax.set_ylabel('$t$ (s)', fontsize=20)
          ax.tick_params(axis='both', which='major', labelsize=16)
          ax.contour(x, y, np.exp(x + 2*y), colors='k', linewidths=0.2, levels=50)
          
          divider = make_axes_locatable(ax)
          cax = divider.append_axes("right", size="2%", pad=0.1)
          cb = fig.colorbar(cf, cax=cax)
          cb.ax.set_title('$T$(\u2103)', fontsize=18)
          ticks = np.linspace(vmin, vmax, 3)
#           print(ticks)
          ticks = np.floor(ticks)
          cb.set_ticks(ticks)
          cb.ax.tick_params(labelsize=16)

          plt.show()

          # fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h), dpi=100, constrained_layout=False)
          # df = pd.DataFrame({'x':x_flat, 't':y_flat, 'T':phi})
          # df.iplot(kind='surface',colorscale='rdylbu')

        

        # save figure
        if self.save_fig:
            fig.savefig(f"fig_{types}.eps", format="eps")
        plt.show()

    def unpack(self, model, params):
        self.model = model
        self.params = params

        self.lb = params["data"]["lb"]
        self.ub = params["data"]["ub"]
        self.n_test = params["data"]["n_test"]
        # self.u_max = params["physic"]["u_max"]
        # self.u_wall = params["physic"]["u_wall"]
        # self.rho = params["physic"]["rho"]
        # self.mu = params["physic"]["mu"]
        self.epochs = params["network"]["epochs"]
        self.coef_bc = params["network"]["coef_bc"]
        self.verboses_adam = params["network"]["verboses_adam"]
        self.verboses_newton = params["network"]["verboses_newton"]
        self.verboses_adaptive = params["network"]["verboses_adaptive"]
    def neumann_loss(self):
        # self.create_test_data()
        x = self.model.x_test
        y = self.model.y_test
        u_pinn = self.model.predict(x, y)
        if self.model.bc_type == 'Neumann':
            u_fea = pd.read_csv('import_test_t2160.csv')
            u_fea = u_fea.to_numpy()
            # T_interp = scipy.interpolate.interp1d(u_fea[:,0],u_fea[:,1])
            # u_test = []
            # for i in self.X_flat:
            #   u_test.append(T_interp(i))
            # u_test = u_fea[:,1:2]+273.0
            u_test = u_fea[:,1:2]

        delta_u = np.abs(u_pinn - u_test)
        #  Absolute error with n_x and n_y points
        abs_err_u = np.sum(delta_u)/(delta_u.shape[0])

        rel_err_u_ij = delta_u/u_test
        # Relative error with n_x and n_y points
        rel_err_u = np.sum(rel_err_u_ij)/(delta_u.shape[0])
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6), dpi=300, constrained_layout=False)
        ax.plot(x, u_pinn, 'blue', label="PINN Solution", linewidth=4)
        ax.plot(u_fea[:,0] , u_test, '--r', label="Finite Element Solution", linewidth=4)
        # ax.plot(u_fea[:,0] , u_test, '--r')
        ax.set_xlabel("x meter", fontsize=15)
        ax.set_ylabel("T (x, t = 2160s) Celcius", fontsize=15)
        ax.grid(linestyle="--")
        ax.legend(fontsize=15)
        ax.set_title("Temperature Comparison", fontsize=20)
        ax.tick_params(axis="both", which='major', labelsize=16)

        # rmse_total = rmse_u
        # print(f"- RMSE x 10^(-2): {rmse_total/(1.e-2):5f}")
        print(f"- Absolute Error: {abs_err_u:5f}")
        print(f"- Relative Error (%): {rel_err_u*100:5f}")


# **RUN**

In [ ]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))


# Set GPU to tensorflow session
with tf.device('/device:GPU:2'):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    # session = tf.InteractiveSession(config=config)
    session = tf.Session(config=config)



## **Channel Flow**

In [ ]:
def material_prop(material):
    if material == "wood":
        # Density (rho) in kg/m^3
        dry_rho = 300 
        rho_ratio = np.array([[1.12],
                              [1.12],
                              [1.0],
                              [1.0],
                              [0.93]])
        rho_temp = np.array([[20.0],
                             [99.0],
                             [120.0],
                             [200.0],
                             [250.0]])
        rho_data = np.concatenate((rho_temp, dry_rho*rho_ratio), axis=1)

        # Thermal conductivity (k) in W/mK
        k_data = np.array([[0.12],
                           [0.15]])
        k_temp = np.array([[20.0],
                           [200.0]])
        k_data = np.concatenate((k_temp, k_data), axis=1)

        # Specific Heat (cp) in J/kgK
        cp_data = np.array([[1.53e+3],
                            [1.77e+3],
                            [13.6e+3],
                            [13.5e+3],
                            [2.12e+3],
                            [2e+3],
                            [1.62e+3]])
        cp_temp = np.array([[20.0],
                            [99.0],
                            [99.001],
                            [120.0],
                            [120.001],
                            [200.0],
                            [250.0]])
        cp_data = np.concatenate((cp_temp, cp_data), axis=1)
    elif material == "steel":
        pass
    return rho_data, k_data, cp_data


def generate_param():
    rho, k, cp = material_prop(material="wood")
    params = {}

    params["data"] = {}
    params["data"]["lb"] = np.array([0., 0.])
    # params["data"]["ub"] = np.array([0.1, 3600.0])
    params["data"]["ub"] = np.array([1.0, 1.0])
    # params["data"]["n_collo"] = 8000
    params["data"]["n_collo"] = 1000
    # params["data"]["n_inlet"] = 301
#     params["data"]["n_inlet"] = 101
    # params["data"]["n_outlet"] = 301
#     params["data"]["n_outlet"] = 101
    # params["data"]["n_wall"] = 201
    params["data"]["n_test"] = 201
    params["data"]["init_wall"] = 25  # Neumann case (in K)
    # params["data"]["init_wall"] = 25 + 273# Neumann case (in K)
    params["data"]["M"] = 500
    # params["data"]["M"] = 3
    params["data"]["left_q"] = 0 # W/m^2
    params["data"]["right_q"] = 300 # W/m^2
    params["data"]["t_check"] = 2160
    params["data"]["seed"] = 543212
    n = 101
    params["data"]["n_wall"] = n
    loc_num = 3
    params["data"]["loc"] = np.linspace(0, 1, loc_num) #in %
    params["data"]["meu"] = [n]*loc_num
    # 0 1 2 3 4 10 30 40 12345
    

    params["physic"] = {}
    params["physic"]["rho"] = rho
    params["physic"]["cp"]  = cp
    params["physic"]["k"]   = k
    # params["physic"]["rho"] = 336  # kg/m^3
    # params["physic"]["cp"]  = 1530 # J/(kg.K)
    # params["physic"]["k"]   = 0.12 # W/(mK)
    # params["physic"]["mu"] = 0.02

    # params["physic"]["T_left"] = 50
    # params["physic"]["T_right"] = 100
    # params["physic"]["T_lower"] = 300
    # params["physic"]["T_upper"] = 150

    params["network"] = {}
    params["network"]["epochs"] = 501 #10001
    params["network"]["coef_bc"] = 1.0
    params["network"]["lr_init"] = 1.0e-3
    params["network"]["beta"] = 0.9
    params["network"]["batches"] = 2000
    params["network"]["verboses_adam"] = 100
    params["network"]["verboses_newton"] = 100
    params["network"]["verboses_adaptive"] = 10
    params["network"]["saver"] = 5000

    return params


#### **Poiseuille Flow - VP**

##### **Generate Parameters**

In [ ]:
BC = 'Dirichlet'
# BC = 'Neumann'
# prob = 'Direct'
prob = 'Inverse'


params = generate_param()

# fun = "relu"
fun = "tanh"
# fun = "swish"
# fun = "sigmoid"
params["network"]["act_fun"] = fun
# params["physic"]["u_max"] = 2.0
# params["physic"]["u_wall"] = 0.0
# params["network"]["layers"] = [2] + 4*[50] + [3]


# params["network"]["layers"] = [2] + 8*[40] + [1]
# params["network"]["layers"] = [2] + 8*[15] + [1]

# best config
# params["network"]["layers"] = [2] + 4*[40] + [1]
# inverse
# params["network"]["layers"] = [2] + 4*[40] + [2]
params["network"]["layers"] = [2] + 2*[5] + [1]
params["data"]["noise"] = 0.03

case = CasesChannel(add_noise=True)
# case.generate_train_data(param=params, bc_type='Neumann')
case.generate_train_data(param=params, bc_type=BC)
case.generate_test_data(param=params)
case.plot()

In [ ]:
x = np.linspace(0,1,101)
y = np.linspace(0,1,101)

X,Y = np.meshgrid(x,y)
x_flat = X.flatten()[:,None]
y_flat = Y.flatten()[:,None]

x = x_flat
t = y_flat
h = np.exp(x + 2*t)

fig_h = 3
fig_w = 5

vmin=min(h)
vmax=max(h)
        
# results.plot_countour(x=X, y=Y, 
#                       x_flat=X_flat, y_flat=Y_flat,
#                       phi=h, phi_=h.reshape(101, 101),
#                       vmin=min(h), vmax=max(h),
#                       fig_w=fig_w, fig_h=fig_h)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h), dpi=300, constrained_layout=False)

# Unpack
cf = ax.scatter(x_flat, y_flat, c=h, 
                alpha=1., edgecolors='none', cmap='jet', marker=".", s=50, vmin=vmin, vmax=vmax)
# ax.set_xlim(self.lb[0], self.ub[0])
# ax.set_ylim(self.lb[1], self.ub[1])

ax.set_title(f'Heat Generation', fontsize=25)

ax.set_xlabel('$x$ (m)', fontsize=20)
ax.set_ylabel('$t$ (s)', fontsize=20)
#         ax.set_xlabel('x', fontsize=20)
#         ax.set_ylabel('t', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.contour(X, Y, h.reshape(101,101), colors='k', linewidths=0.2, levels=50)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.1)
cb = fig.colorbar(cf, cax=cax)
ticks = np.linspace(vmin, vmax, 3)
ticks = np.round(ticks, 2)
ticks[0] = ticks[0] + 0.01
cb.set_ticks(ticks)
cb.ax.set_title('$q$[W/m3]',fontsize=15)
cb.ax.tick_params(labelsize=15)

##### **Run**

In [ ]:
# Create Model Instance
model = PinnVP(data=case.data, adaptive_model=False,
               params=params, bc_type=BC, problem_type=prob)
              #  exist_model=True, file_dir='/content/poiseuille-vp.pickle')

In [ ]:
# Fit using ADAM
model.fit_newton()



In [ ]:
lambda_log = np.array(model.lambda_1_log)[:,0]
x1 = np.linspace(1, len(lambda_log), len(lambda_log))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6), constrained_layout=True, dpi=300)
ax.plot(x1, lambda_log, "r", linestyle="solid", label="Lambda")
# ax.plot(x2, lambda_2_log, "g", linestyle="dashdot", label="Lambda 2")

ax.set_xlim(0, x1[-1])
ax.set_xlabel("Iterations", fontsize=12)
ax.set_ylabel("Lambda Value", fontsize=12)
ax.grid(linestyle="--")
ax.legend(fontsize=10)
ax.set_title('Lambda History', fontsize=17)
ax.tick_params(axis='both', which='major', labelsize=10)

In [ ]:
# tf.trainable_variables()

In [ ]:
# Create Results
results = PostChannel(model=model,
                      params=params,
                      save_fig=False)

results.display_loss()
results.display_contour()

# Save model
# model.save_model('test-seed=1-after.pickle')

In [ ]:
x, y, u_pinn, u_test = results.calculate_error(results.n_test)

In [ ]:
# Save model
model.save_model('1_inverse_param.pickle')

In [ ]:
n = 100
x = np.ones((n,1))*1
t = lhs(1,n)*1
# T = np.exp(1+2*(t/self.ub[1]))
T = np.exp(x+2*t)

mu = T
noise = 0.10
sigma = noise * mu
np.random.seed(54321)
T_new = np.random.normal(mu, sigma).reshape(mu.shape)
T_neww = T + np.random.normal(0, noise*max(T),n).reshape(T.shape)
T_newww = T + noise*np.std(T)*np.random.randn(T.shape[0],T.shape[1])



In [ ]:
print(np.mean(abs(T-T_new)))
print(np.mean(abs(T-T_neww)))
print(np.mean(abs(T-T_newww)))

In [ ]:
print(np.std(abs(T-T_new)))
print(np.std(abs(T-T_neww)))
print(np.std(abs(T-T_newww)))

In [ ]:
print(np.std(abs(T-T_new)) / np.mean(abs(T-T_new)))
print(np.std(abs(T-T_neww)) / np.mean(abs(T-T_neww)))
print(np.std(abs(T-T_newww)) / np.mean(abs(T-T_newww)))


In [ ]:
# def main_loop(layers, neurons, noise, BC, prob, seed):
#     t = timeit.default_timer()
#     print(noise)
#     params = generate_param()
#     print(noise)
#     params["network"]["layers"] = [2] + layers*[neurons] + [1]
#     params["data"]["noise"] = noise
#     params["data"]["seed"] = seed

#     # case = CasesChannel()
#     case = CasesChannel(add_noise=True)
#     case.generate_train_data(param=params, bc_type=BC)
#     case.generate_test_data(param=params)

#     model = PinnVP(data=case.data, adaptive_model=False,
#                  params=params, bc_type=BC, problem_type=prob)

#     model.fit_newton()

#     results = PostChannel(model=model,
#                       params=params,
#                       save_fig=False)

#     results.display_loss()
#     #   case
#     results.display_contour()
#     elapsed = timeit.default_timer() - t

#     if prob == "Direct":
#         return results.abs_err_u, results.rel_err_u
#     else:
#         return model.lambda_1_log[-1][0][0], elapsed


# # RUN
# if __name__ == "__main__": 
#     layers = [2] # num of hidden layers
#     neurons = [5] # num of neurons
#     noises = [0.00, 0.01, 0.03] #, 0.15] # noise level
#     seed = [543212345, 2345, 5432123, 543212, 54321, 345, 543, 54, 5, 4] # num of hidden layers
#     # N_u = [500, 1000, 1500, 2000] # num of training data 

#     abs_err_table = np.zeros((len(layers), len(neurons)))
#     rel_err_table = np.zeros((len(layers), len(neurons)))

#     lambda_1_table1 = np.zeros((len(noises),len(seed)))
#     lambda_1_table2 = np.zeros(len(noises))
#     time_table = np.zeros((len(noises), len(seed)))

#     # for i in range(len(layers)):
#     #   for j in range(len(neurons)):
#     #       abs_err_table[i,j], rel_err_table[i,j] = main_loop(layers[i], neurons[j], noises[0], BC="Dirichlet", prob="Direct")

#     # np.savetxt('./abs_err_table.csv', abs_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')
#     # np.savetxt('./rel_err_table.csv', rel_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')

# #     for i in range(len(noises)):
#     for j in range(0, len(seed)):
#         lambda_1_table1[i, j], time_table[i,j] =  main_loop(2, 5, noises[0], BC="Dirichlet", prob="Inverse", seed=seed[j])
#     #     lambda_1_table2[i] =  main_loop(6, 15, noises[i], BC="Dirichlet", prob="Inverse")

#     np.savetxt('./lambda_1_table1.csv', lambda_1_table1, delimiter=' ', fmt='%2.6f', newline=' \\\n')
#     np.savetxt('./time_table.csv', time_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')

#     #   np.savetxt('./lambda_1_table2.csv', lambda_1_table2, delimiter=' ', fmt='%2.6f', newline=' \\\n')



In [ ]:
# relu
def main_loop(layers, neurons, noise, BC, prob, seed):
    t = timeit.default_timer()
    print(noise)
    params = generate_param()
    print(seed)
    params["network"]["layers"] = [2] + layers*[neurons] + [1]
    params["data"]["noise"] = noise
    params["data"]["seed"] = seed
    loc_num = 4
    n = 101
    params["data"]["n_wall"] = n
    params["data"]["loc"] = np.linspace(0, 1, loc_num) #in %
    params["data"]["meu"] = [n]*loc_num
    
#     fun = "relu"
    fun = "tanh"
    # fun = "swish"
    # fun = "sigmoid"
    params["network"]["act_fun"] = fun

    # case = CasesChannel()
    case = CasesChannel(add_noise=True)
    case.generate_train_data(param=params, bc_type=BC)
    case.generate_test_data(param=params)
    case.plot()

    model = PinnVP(data=case.data, adaptive_model=False,
                 params=params, bc_type=BC, problem_type=prob)

    model.fit_newton()
    model_list.append(model)
    

    results = PostChannel(model=model,
                      params=params,
                      save_fig=False)

    results.display_loss()
    results_list.append(results)
    
#     lambda_log = []
    lambda_log = (np.array(model.lambda_1_log)[:,0])
    x = np.linspace(1, len(lambda_log), len(lambda_log))
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6), constrained_layout=True, dpi=300)
    ax.plot(x, lambda_log, "r", linestyle="solid", label="Noise free")
    ax.set_xlabel("Iterations", fontsize=12)
    ax.set_ylabel("Lambda Value", fontsize=12)
    ax.grid(linestyle="--")
    ax.legend(fontsize=10)
    ax.set_title('Lambda History', fontsize=17)
    ax.tick_params(axis='both', which='major', labelsize=10)


    #   case
    results.display_contour()
    elapsed = timeit.default_timer() - t

    if prob == "Direct":
        return results.abs_err_u, results.rel_err_u
    else:
        return model.lambda_1_log[-1][0][0], elapsed


# RUN
if __name__ == "__main__": 
    layers = [2] # num of hidden layers
    neurons = [5] # num of neurons
    noises = [0.00, 0.01, 0.03] #, 0.15] # noise level
    seed = [4]*3 #,2345, 5432123, 543212, 54321, 345, 543, 54, 5, 4] # num of hidden layers
    # N_u = [500, 1000, 1500, 2000] # num of training data 

#     abs_err_table = np.zeros((len(layers), len(neurons)))
#     rel_err_table = np.zeros((len(layers), len(neurons)))

    lambda_1_table1 = np.zeros((len(noises),len(seed)))
#     lambda_1_table2 = np.zeros(len(noises))
    time_table = np.zeros((len(noises),len(seed)))
    model_list = []
    results_list = []

    # for i in range(len(layers)):
    #   for j in range(len(neurons)):
    #       abs_err_table[i,j], rel_err_table[i,j] = main_loop(layers[i], neurons[j], noises[0], BC="Dirichlet", prob="Direct")

    # np.savetxt('./abs_err_table.csv', abs_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    # np.savetxt('./rel_err_table.csv', rel_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')

    for i in range(len(noises)):
#         for j in range(0, len(seed)):
            lambda_1_table1[i, 0], time_table[i, 0] =  main_loop(2, 5, noises[i], BC="Dirichlet", 
                                                                 prob="Inverse", seed=seed[i])
    #     lambda_1_table2[i] =  main_loop(6, 15, noises[i], BC="Dirichlet", prob="Inverse")
    
    

#     np.savetxt('./lambda_1_table1_ReLU.csv', lambda_1_table1, delimiter=' ', fmt='%2.6f', newline=' \\\n')
#     np.savetxt('./time_table_ReLU.csv', time_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    
#     lambda_1_log = []
#     for i in range(0, len(model_list)):
#         model_list[i].save_model(f"noise_0%_seed_{seed[i]}")
#         lambda_1_log.append(np.array(model_list[i].lambda_1_log)[:,0].flatten()[:,None])
    
#         np.savetxt(f'./lambda_1_log_noise_0%_seed_{seed[i]}.csv', lambda_1_log, delimiter=' ', fmt='%2.6f', newline=' \\\n')
        
#     for i in range(len(lambda_log)):

       

    #   np.savetxt('./lambda_1_table2.csv', lambda_1_table2, delimiter=' ', fmt='%2.6f', newline=' \\\n')



In [ ]:
lambda_pred_tanh = results_list[0].u_pinn.reshape(201,201)
lambda_ref_tanh = results_list[0].u_test.reshape(201,201)

lambda_pred_relu = results_list[1].u_pinn.reshape(201,201)
lambda_ref_relu = results_list[1].u_test.reshape(201,201)

lambda_pred_swish = results_list[2].u_pinn.reshape(201,201)
lambda_ref_swish = results_list[2].u_test.reshape(201,201)


In [ ]:
fig_w = 10
fig_h = 6

y = np.linspace(0, 1, 201)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(fig_w, fig_h), dpi=100, constrained_layout=False)
        
# print(phi_pinn)
ax[0].plot(y, lambda_pred_tanh[:,100].flatten(), 'b', label="PINN solution", linewidth=4)
# ax[1].plot(y, phi_analytics[:,self.n_x//2], '--r', label="Analytical solution", linewidth=4)
ax[0].plot(y, lambda_ref_tanh[:,100].flatten(), '--r', label="Exact solution", linewidth=4)


x_lim_min = min(np.min(lambda_pred_tanh), np.min(lambda_ref_tanh))
x_lim_max = max(np.max(lambda_pred_tanh), np.max(lambda_ref_tanh))
# x_lim_min = 0
# x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
ax[0].set_ylim(x_lim_min, x_lim_max)
ax[0].set_xlim(0., 1.0)

#ax.set_xlabel("$u$ (m/s)", fontsize=20)
#ax.set_ylabel("$y$ (m)", fontsize=20)
ax[0].set_xlabel("$t$ (s), $x=0.5$ m", fontsize=25)
# ax[1].set_ylabel("T (x = 0.5, t)", fontsize=15)
ax[0].set_ylabel("$T$ (\u2103)", fontsize=25)
ax[0].grid(linestyle="--")
ax[0].legend(fontsize=14)
ax[0].set_title("0\% Noise", fontsize=35)
# ax[1].set_title("Temperature comparison", fontsize=20)
ax[0].tick_params(axis='both', which='major', labelsize=23)

ax[1].plot(y, lambda_pred_relu[:,100].flatten(), 'b', label="PINN solution", linewidth=4)
# ax[0].plot(y, phi_analytics[:,self.n_x//4], '--r', label="Analytical solution", linewidth=4)
ax[1].plot(y, lambda_ref_relu[:,100].flatten(), '--r', label="Exact solution", linewidth=4)

# x_lim_min = min(min(phi_pinn[:,1:2].flatten()), min(phi_analytics[:,1:2]))
# x_lim_max = max(max(phi_pinn[:,1:2].flatten()), max(phi_analytics[:,1:2]))
# x_lim_min = x_lim_min - 0.2*abs(x_lim_min)
# x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
# ax[0].set_ylim(x_lim_min, x_lim_max)
# ax[0].set_xlim(self.lb[1], self.ub[1])

x_lim_min = min(np.min(lambda_pred_relu), np.min(lambda_ref_relu))
x_lim_max = max(np.max(lambda_pred_relu), np.max(lambda_ref_relu))
# x_lim_min = 0
# x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
ax[1].set_ylim(x_lim_min, x_lim_max)
ax[1].set_xlim(0., 1.0)
#ax.set_xlabel("$u$ (m/s)", fontsize=20)
#ax.set_ylabel("$y$ (m)", fontsize=20)
ax[1].set_xlabel("$t$ (s), $x=0.5$ m", fontsize=25)
# ax[1].set_ylabel("T (x = 0.5, t)", fontsize=15)
ax[1].set_ylabel("$T$ (\u2103)", fontsize=25)
ax[1].grid(linestyle="--")
ax[1].legend(fontsize=14)
ax[1].set_title("1\% Noise", fontsize=35)
# ax[0].set_title("Temperature comparison", fontsize=20)
ax[1].tick_params(axis='both', which='major', labelsize=23)

ax[2].plot(y, lambda_pred_swish[:,100].flatten(), 'b', label="PINN solution", linewidth=4)
# ax[2].plot(y, phi_analytics[:,self.n_x//4*3+1], '--r', label="Analytical solution", linewidth=4)
ax[2].plot(y, lambda_ref_swish[:,100].flatten(), '--r', label="Exact solution", linewidth=4)

# x_lim_min = min(min(phi_pinn[:,2:3].flatten()), min(phi_analytics[:,2:3]))
# x_lim_max = max(max(phi_pinn[:,2:3].flatten()), max(phi_analytics[:,2:3]))
# x_lim_min = x_lim_min - 0.2*abs(x_lim_min)
# x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
# ax[2].set_ylim(x_lim_min, x_lim_max)
# ax[2].set_xlim(self.lb[1], self.ub[1])

x_lim_min = min(np.min(lambda_pred_swish), np.min(lambda_ref_swish))
x_lim_max = max(np.max(lambda_pred_swish), np.max(lambda_ref_swish))
# x_lim_min = 0
# x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
ax[2].set_ylim(x_lim_min, x_lim_max)
ax[2].set_xlim(0., 1.0)
#ax.set_xlabel("$u$ (m/s)", fontsize=20)
#ax.set_ylabel("$y$ (m)", fontsize=20)
ax[2].set_xlabel("$t$ (s), $x=0.5$ m", fontsize=25)
# ax[1].set_ylabel("T (x = 0.5, t)", fontsize=15)
ax[2].set_ylabel("$T$ (\u2103)", fontsize=25)
ax[2].grid(linestyle="--")
ax[2].legend(fontsize=14)
ax[2].set_title("3\% Noise", fontsize=35)
# ax[2].set_title("Temperature comparison", fontsize=20)
ax[2].tick_params(axis='both', which='major', labelsize=23)
plt.tight_layout()

In [ ]:
##### fig_w = 10
fig_h = 6

y = np.linspace(0, 1, 201)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(fig_w, fig_h), dpi=100, constrained_layout=False)
        
# print(phi_pinn)
ax[0].plot(y, lambda_pred_tanh[100,:].flatten(), 'b', label="PINN solution", linewidth=4)
# ax[1].plot(y, phi_analytics[:,self.n_x//2], '--r', label="Analytical solution", linewidth=4)
ax[0].plot(y, lambda_ref_tanh[100,:].flatten(), '--r', label="Exact solution", linewidth=4)


x_lim_min = min(np.min(lambda_pred_tanh), np.min(lambda_ref_tanh))
x_lim_max = max(np.max(lambda_pred_tanh), np.max(lambda_ref_tanh))
# x_lim_min = 0
# x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
ax[0].set_ylim(x_lim_min, x_lim_max)
ax[0].set_xlim(0., 1.0)

#ax.set_xlabel("$u$ (m/s)", fontsize=20)
#ax.set_ylabel("$y$ (m)", fontsize=20)
ax[0].set_xlabel("$x$ (m), $t=0.5$ s", fontsize=25)
# ax[1].set_ylabel("T (x = 0.5, t)", fontsize=15)
ax[0].set_ylabel("$T$ (\u2103)", fontsize=25)
ax[0].grid(linestyle="--")
ax[0].legend(fontsize=14)
ax[0].set_title("0\% Noise", fontsize=35)
# ax[1].set_title("Temperature comparison", fontsize=20)
ax[0].tick_params(axis='both', which='major', labelsize=23)

ax[1].plot(y, lambda_pred_relu[100,:].flatten(), 'b', label="PINN solution", linewidth=4)
# ax[0].plot(y, phi_analytics[:,self.n_x//4], '--r', label="Analytical solution", linewidth=4)
ax[1].plot(y, lambda_ref_relu[100,:].flatten(), '--r', label="Exact solution", linewidth=4)

# x_lim_min = min(min(phi_pinn[:,1:2].flatten()), min(phi_analytics[:,1:2]))
# x_lim_max = max(max(phi_pinn[:,1:2].flatten()), max(phi_analytics[:,1:2]))
# x_lim_min = x_lim_min - 0.2*abs(x_lim_min)
# x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
# ax[0].set_ylim(x_lim_min, x_lim_max)
# ax[0].set_xlim(self.lb[1], self.ub[1])

x_lim_min = min(np.min(lambda_pred_relu), np.min(lambda_ref_relu))
x_lim_max = max(np.max(lambda_pred_relu), np.max(lambda_ref_relu))
# x_lim_min = 0
# x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
ax[1].set_ylim(x_lim_min, x_lim_max)
ax[1].set_xlim(0., 1.0)
#ax.set_xlabel("$u$ (m/s)", fontsize=20)
#ax.set_ylabel("$y$ (m)", fontsize=20)
ax[1].set_xlabel("$x$ (m), $t=0.5$ s", fontsize=25)
# ax[1].set_ylabel("T (x = 0.5, t)", fontsize=15)
ax[1].set_ylabel("$T$ (\u2103)", fontsize=25)
ax[1].grid(linestyle="--")
ax[1].legend(fontsize=14)
ax[1].set_title("1\% Noise", fontsize=35)
# ax[0].set_title("Temperature comparison", fontsize=20)
ax[1].tick_params(axis='both', which='major', labelsize=23)

ax[2].plot(y, lambda_pred_swish[100,:].flatten(), 'b', label="PINN solution", linewidth=4)
# ax[2].plot(y, phi_analytics[:,self.n_x//4*3+1], '--r', label="Analytical solution", linewidth=4)
ax[2].plot(y, lambda_ref_swish[100,:].flatten(), '--r', label="Exact solution", linewidth=4)

# x_lim_min = min(min(phi_pinn[:,2:3].flatten()), min(phi_analytics[:,2:3]))
# x_lim_max = max(max(phi_pinn[:,2:3].flatten()), max(phi_analytics[:,2:3]))
# x_lim_min = x_lim_min - 0.2*abs(x_lim_min)
# x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
# ax[2].set_ylim(x_lim_min, x_lim_max)
# ax[2].set_xlim(self.lb[1], self.ub[1])

x_lim_min = min(np.min(lambda_pred_swish), np.min(lambda_ref_swish))
x_lim_max = max(np.max(lambda_pred_swish), np.max(lambda_ref_swish))
# x_lim_min = 0
# x_lim_max = x_lim_max + 0.2*abs(x_lim_max)
ax[2].set_ylim(x_lim_min, x_lim_max)
ax[2].set_xlim(0., 1.0)
#ax.set_xlabel("$u$ (m/s)", fontsize=20)
#ax.set_ylabel("$y$ (m)", fontsize=20)
ax[2].set_xlabel("$x$ (m), $t=0.5$ s", fontsize=25)
# ax[1].set_ylabel("T (x = 0.5, t)", fontsize=15)
ax[2].set_ylabel("$T$ (\u2103)", fontsize=25)
ax[2].grid(linestyle="--")
ax[2].legend(fontsize=14)
ax[2].set_title("3\% Noise", fontsize=35)
# ax[2].set_title("Temperature comparison", fontsize=20)
ax[2].tick_params(axis='both', which='major', labelsize=23)
plt.tight_layout()

In [ ]:
# swish
def main_loop(layers, neurons, noise, BC, prob, seed):
    t = timeit.default_timer()
    print(noise)
    params = generate_param()
    print(seed)
    params["network"]["layers"] = [2] + layers*[neurons] + [1]
    params["data"]["noise"] = noise
    params["data"]["seed"] = seed
    loc_num = 4
    n = 101
    params["data"]["n_wall"] = n
    params["data"]["loc"] = np.linspace(0, 1, loc_num) #in %
    params["data"]["meu"] = [n]*loc_num
    
#     fun = "relu"
#     fun = "tanh"
    fun = "swish"
    # fun = "sigmoid"
    params["network"]["act_fun"] = fun

    # case = CasesChannel()
    case = CasesChannel(add_noise=True)
    case.generate_train_data(param=params, bc_type=BC)
    case.generate_test_data(param=params)
    case.plot()

    model = PinnVP(data=case.data, adaptive_model=False,
                 params=params, bc_type=BC, problem_type=prob)

    model.fit_newton()
    model_list.append(model)
    

    results = PostChannel(model=model,
                      params=params,
                      save_fig=False)

    results.display_loss()
    
#     lambda_log = []
    lambda_log = (np.array(model.lambda_1_log)[:,0])
    x = np.linspace(1, len(lambda_log), len(lambda_log))
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6), constrained_layout=True, dpi=300)
    ax.plot(x, lambda_log, "r", linestyle="solid", label="Noise free")
    ax.set_xlabel("Iterations", fontsize=12)
    ax.set_ylabel("Lambda Value", fontsize=12)
    ax.grid(linestyle="--")
    ax.legend(fontsize=10)
    ax.set_title('Lambda History', fontsize=17)
    ax.tick_params(axis='both', which='major', labelsize=10)


    #   case
    results.display_contour()
    elapsed = timeit.default_timer() - t

    if prob == "Direct":
        return results.abs_err_u, results.rel_err_u
    else:
        return model.lambda_1_log[-1][0][0], elapsed


# RUN
if __name__ == "__main__": 
    layers = [2] # num of hidden layers
    neurons = [5] # num of neurons
    noises = [0.00, 0.01, 0.03] #, 0.15] # noise level
    seed = [543212345, 2345, 5432123, 543212, 54321, 345, 543, 54, 5, 4] # num of hidden layers
    # N_u = [500, 1000, 1500, 2000] # num of training data 

#     abs_err_table = np.zeros((len(layers), len(neurons)))
#     rel_err_table = np.zeros((len(layers), len(neurons)))

    lambda_1_table1 = np.zeros((len(noises),len(seed)))
#     lambda_1_table2 = np.zeros(len(noises))
    time_table = np.zeros((len(noises),len(seed)))
    model_list = []

    # for i in range(len(layers)):
    #   for j in range(len(neurons)):
    #       abs_err_table[i,j], rel_err_table[i,j] = main_loop(layers[i], neurons[j], noises[0], BC="Dirichlet", prob="Direct")

    # np.savetxt('./abs_err_table.csv', abs_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    # np.savetxt('./rel_err_table.csv', rel_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')

    for i in range(len(noises)):
        for j in range(0, len(seed)):
            lambda_1_table1[i, j], time_table[i, j] =  main_loop(2, 5, noises[i], BC="Dirichlet", prob="Inverse", seed=seed[j])
    #     lambda_1_table2[i] =  main_loop(6, 15, noises[i], BC="Dirichlet", prob="Inverse")
    
    

    np.savetxt('./lambda_1_table1_swish.csv', lambda_1_table1, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    np.savetxt('./time_table_swish.csv', time_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    
#     lambda_1_log = []
#     for i in range(0, len(model_list)):
#         model_list[i].save_model(f"noise_0%_seed_{seed[i]}")
#         lambda_1_log.append(np.array(model_list[i].lambda_1_log)[:,0].flatten()[:,None])
    
#         np.savetxt(f'./lambda_1_log_noise_0%_seed_{seed[i]}.csv', lambda_1_log, delimiter=' ', fmt='%2.6f', newline=' \\\n')
        
#     for i in range(len(lambda_log)):

       

    #   np.savetxt('./lambda_1_table2.csv', lambda_1_table2, delimiter=' ', fmt='%2.6f', newline=' \\\n')



In [ ]:
# sigmoid
def main_loop(layers, neurons, noise, BC, prob, seed):
    t = timeit.default_timer()
    print(noise)
    params = generate_param()
    print(seed)
    params["network"]["layers"] = [2] + layers*[neurons] + [1]
    params["data"]["noise"] = noise
    params["data"]["seed"] = seed
    loc_num = 4
    n = 101
    params["data"]["n_wall"] = n
    params["data"]["loc"] = np.linspace(0, 1, loc_num) #in %
    params["data"]["meu"] = [n]*loc_num
    
#     fun = "relu"
#     fun = "tanh"
#     fun = "swish"
    fun = "sigmoid"
    params["network"]["act_fun"] = fun

    # case = CasesChannel()
    case = CasesChannel(add_noise=True)
    case.generate_train_data(param=params, bc_type=BC)
    case.generate_test_data(param=params)
    case.plot()

    model = PinnVP(data=case.data, adaptive_model=False,
                 params=params, bc_type=BC, problem_type=prob)

    model.fit_newton()
    model_list.append(model)
    

    results = PostChannel(model=model,
                      params=params,
                      save_fig=False)

    results.display_loss()
    
#     lambda_log = []
    lambda_log = (np.array(model.lambda_1_log)[:,0])
    x = np.linspace(1, len(lambda_log), len(lambda_log))
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6), constrained_layout=True, dpi=300)
    ax.plot(x, lambda_log, "r", linestyle="solid", label="Noise free")
    ax.set_xlabel("Iterations", fontsize=12)
    ax.set_ylabel("Lambda Value", fontsize=12)
    ax.grid(linestyle="--")
    ax.legend(fontsize=10)
    ax.set_title('Lambda History', fontsize=17)
    ax.tick_params(axis='both', which='major', labelsize=10)


    #   case
    results.display_contour()
    elapsed = timeit.default_timer() - t

    if prob == "Direct":
        return results.abs_err_u, results.rel_err_u
    else:
        return model.lambda_1_log[-1][0][0], elapsed


# RUN
if __name__ == "__main__": 
    layers = [2] # num of hidden layers
    neurons = [5] # num of neurons
    noises = [0.00, 0.01, 0.03] #, 0.15] # noise level
    seed = [543212345, 2345, 5432123, 543212, 54321, 345, 543, 54, 5, 4] # num of hidden layers
    # N_u = [500, 1000, 1500, 2000] # num of training data 

#     abs_err_table = np.zeros((len(layers), len(neurons)))
#     rel_err_table = np.zeros((len(layers), len(neurons)))

    lambda_1_table1 = np.zeros((len(noises),len(seed)))
#     lambda_1_table2 = np.zeros(len(noises))
    time_table = np.zeros((len(noises),len(seed)))
    model_list = []

    # for i in range(len(layers)):
    #   for j in range(len(neurons)):
    #       abs_err_table[i,j], rel_err_table[i,j] = main_loop(layers[i], neurons[j], noises[0], BC="Dirichlet", prob="Direct")

    # np.savetxt('./abs_err_table.csv', abs_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    # np.savetxt('./rel_err_table.csv', rel_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')

    for i in range(len(noises)):
        for j in range(0, len(seed)):
            lambda_1_table1[i, j], time_table[i, j] =  main_loop(2, 5, noises[i], BC="Dirichlet", prob="Inverse", seed=seed[j])
    #     lambda_1_table2[i] =  main_loop(6, 15, noises[i], BC="Dirichlet", prob="Inverse")
    
    

    np.savetxt('./lambda_1_table1_sigmoid.csv', lambda_1_table1, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    np.savetxt('./time_table_sigmoid.csv', time_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    
#     lambda_1_log = []
#     for i in range(0, len(model_list)):
#         model_list[i].save_model(f"noise_0%_seed_{seed[i]}")
#         lambda_1_log.append(np.array(model_list[i].lambda_1_log)[:,0].flatten()[:,None])
    
#         np.savetxt(f'./lambda_1_log_noise_0%_seed_{seed[i]}.csv', lambda_1_log, delimiter=' ', fmt='%2.6f', newline=' \\\n')
        
#     for i in range(len(lambda_log)):

       

    #   np.savetxt('./lambda_1_table2.csv', lambda_1_table2, delimiter=' ', fmt='%2.6f', newline=' \\\n')



In [ ]:
def main_loop(layers, neurons, noise, BC, prob, seed):
    t = timeit.default_timer()
    print(noise)
    params = generate_param()
#     print(noise)
    params["network"]["layers"] = [2] + layers*[neurons] + [1]
    params["data"]["noise"] = noise
    params["data"]["seed"] = seed
    
    loc_num = 4
    n = 101
    params["data"]["n_wall"] = n
    params["data"]["loc"] = np.linspace(0, 1, loc_num) #in %
    params["data"]["meu"] = [n]*loc_num
    
#     fun = "relu"
    fun = "tanh"
#     fun = "swish"
#     fun = "sigmoid"
    params["network"]["act_fun"] = fun

    # case = CasesChannel()
    case = CasesChannel(add_noise=True)
    case.generate_train_data(param=params, bc_type=BC)
    case.generate_test_data(param=params)

    model = PinnVP(data=case.data, adaptive_model=False,
                 params=params, bc_type=BC, problem_type=prob)

    model.fit_newton()
    model_list.append(model)
    

    results = PostChannel(model=model,
                      params=params,
                      save_fig=False)

    results.display_loss()
    #   case
    results.display_contour()
    elapsed = timeit.default_timer() - t
    lambda_log.append(model.lambda_1_log)

    if prob == "Direct":
        return results.abs_err_u, results.rel_err_u
    else:
        return model.lambda_1_log[-1][0][0], elapsed


# RUN
if __name__ == "__main__": 
    layers = [2] # num of hidden layers
    neurons = [5] # num of neurons
    noises = [0.00, 0.01, 0.03] # noise level
    seed = [12345] #, 2345, 5432123, 543212, 54321, 345, 543, 54, 5, 4] # num of hidden layers
    # N_u = [500, 1000, 1500, 2000] # num of training data 

    abs_err_table = np.zeros((len(layers), len(neurons)))
    rel_err_table = np.zeros((len(layers), len(neurons)))

    lambda_1_table1 = np.zeros((len(noises),len(seed)))
    lambda_1_table2 = np.zeros(len(noises))
    time_table = np.zeros((len(noises), len(seed)))
    model_list = []
    lambda_log = []

    for i in range(len(noises)):
        for j in range(0, len(seed)):
            lambda_1_table1[i, j], time_table[i,j] =  main_loop(2, 5, noises[i], BC="Dirichlet", prob="Inverse", seed=seed[j])
    #     lambda_1_table2[i] =  main_loop(6, 15, noises[i], BC="Dirichlet", prob="Inverse")
    
    for i in range(len(model_list)):
        model_list[i].save_model(f'model_noise_{noises[i]}.pickle')
        lambda_1_log = np.array(model_list[i].lambda_1_log)[:,0].flatten()[:,None]


In [ ]:
for i in range(len(lambda_log)):
    
    np.savetxt(f'./lambda_1_log_noise_{noises[i]}.csv', np.array(lambda_log[i])[:,0], delimiter=' ', fmt='%2.6f', newline=' \\\n')


In [ ]:
noise = [0.00, 0.01, 0.03]
lambda_ = []
for i in noise:
    lambda_.append(pd.read_csv(f"lambda_1_log_noise_{i}.csv",header=None).to_numpy())
    

x0 = np.linspace(0, len(lambda_[0][:,0]), len(lambda_[0][:,0]))
x1 = np.linspace(0, len(lambda_[1][:,0]), len(lambda_[1][:,0]))
x2 = np.linspace(0, len(lambda_[2][:,0]), len(lambda_[2][:,0]))
# x3 = np.linspace(0, len(lambda_[3][:,0]), len(lambda_[3][:,0]))

lambda_new = []
for i in range(0,len(noise)):
    lambda_edited = np.zeros((len(lambda_[i]),1))
    for j in range(0, len(lambda_[i])):
        lambda_edited[j,0] = lambda_[i][j][0].split(" ")[0]
    lambda_new.append(lambda_edited)

In [ ]:
# base = pd.read_csv("baseline_test.csv").to_numpy()

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 8), constrained_layout=True, dpi=300)
# ax.plot(base[:,0], base[:,1], "k", linestyle="solid", label="Baseline")
ax.plot(x0, lambda_new[0][:,0], "r", linestyle="solid", label="Noise free")
ax.plot(x1, lambda_new[1][:,0], "b", linestyle="dashed", label="1\% noise")
ax.plot(x2, lambda_new[2][:,0], "g", linestyle="dashdot", label="3\% noise")
# ax.plot(x3, lambda_[3][:,0], "c", linestyle="dotted", label="15\% noise")
# ax.plot(M[4][:,0], M[4][:,1], "purple", linestyle="-.", label="M = 10000")

# ax.set_xlim(0, x1[-1])
# ax.set_yscale("log")
ax.set_xlabel("Iterations", fontsize=30)
ax.set_ylabel("$\lambda$", fontsize=30)
ax.grid(linestyle="--")
ax.legend(fontsize=25)
ax.set_title('$\lambda$ History', fontsize=40)
ax.tick_params(axis='both', which='major', labelsize=25)

In [ ]:
lambda_new = []
for i in range(0,len(noise)):
    lambda_edited = np.zeros((len(lambda_[i]),1))
    for j in range(0, len(lambda_[i])):
        lambda_edited[j,0] = lambda_[i][j][0].split(" ")[0]
    lambda_new.append(lambda_edited)

In [ ]:
float(lambda_[0][5][0].split(" ")[0])

In [ ]:
(lambda_new)[0][0]

In [ ]:
# base = pd.read_csv("baseline_test.csv").to_numpy()

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 8), constrained_layout=True, dpi=300)
# ax.plot(base[:,0], base[:,1], "k", linestyle="solid", label="Baseline")
ax.plot(x0, lambda_new[0][:,0], "r", linestyle="solid", label="Noise free")
ax.plot(x1, lambda_new[1][:,0], "b", linestyle="dashed", label="1\% noise")
ax.plot(x2, lambda_new[2][:,0], "g", linestyle="dashdot", label="3\% noise")
# ax.plot(x3, lambda_[3][:,0], "c", linestyle="dotted", label="15\% noise")
# ax.plot(M[4][:,0], M[4][:,1], "purple", linestyle="-.", label="M = 10000")

# ax.set_xlim(0, x1[-1])
# ax.set_yscale("log")
ax.set_xlabel("Iterations", fontsize=30)
ax.set_ylabel("$\lambda$", fontsize=30)
ax.grid(linestyle="--")
ax.legend(fontsize=25)
ax.set_title('$\lambda$ History', fontsize=40)
ax.tick_params(axis='both', which='major', labelsize=25)

In [ ]:
lambda_new

In [ ]:
# 51 n
def main_loop(layers, neurons, noise, BC, prob, seed):
    t = timeit.default_timer()
    print(noise)
    params = generate_param()
    print(seed)
    params["network"]["layers"] = [2] + layers*[neurons] + [1]
    params["data"]["noise"] = noise
    params["data"]["seed"] = seed
    loc_num = 4
    n = 51
    params["data"]["n_wall"] = n
    params["data"]["loc"] = np.linspace(0, 1, loc_num) #in %
    params["data"]["meu"] = [n]*loc_num

    # case = CasesChannel()
    case = CasesChannel(add_noise=True)
    case.generate_train_data(param=params, bc_type=BC)
    case.generate_test_data(param=params)
    case.plot()

    model = PinnVP(data=case.data, adaptive_model=False,
                 params=params, bc_type=BC, problem_type=prob)

    model.fit_newton()
    model_list.append(model)
    

    results = PostChannel(model=model,
                      params=params,
                      save_fig=False)

    results.display_loss()
    
#     lambda_log = []
    lambda_log = (np.array(model.lambda_1_log)[:,0])
    x = np.linspace(1, len(lambda_log), len(lambda_log))
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6), constrained_layout=True, dpi=300)
    ax.plot(x, lambda_log, "r", linestyle="solid", label="Noise free")
    ax.set_xlabel("Iterations", fontsize=12)
    ax.set_ylabel("Lambda Value", fontsize=12)
    ax.grid(linestyle="--")
    ax.legend(fontsize=10)
    ax.set_title('Lambda History', fontsize=17)
    ax.tick_params(axis='both', which='major', labelsize=10)


    #   case
    results.display_contour()
    elapsed = timeit.default_timer() - t

    if prob == "Direct":
        return results.abs_err_u, results.rel_err_u
    else:
        return model.lambda_1_log[-1][0][0], elapsed


# RUN
if __name__ == "__main__": 
    layers = [2] # num of hidden layers
    neurons = [5] # num of neurons
    noises = [0.00, 0.01, 0.03] #, 0.15] # noise level
    seed = [543212345, 2345, 5432123, 543212, 54321, 345, 543, 54, 5, 4] # num of hidden layers
    # N_u = [500, 1000, 1500, 2000] # num of training data 

#     abs_err_table = np.zeros((len(layers), len(neurons)))
#     rel_err_table = np.zeros((len(layers), len(neurons)))

    lambda_1_table1 = np.zeros((len(noises),len(seed)))
#     lambda_1_table2 = np.zeros(len(noises))
    time_table = np.zeros((len(noises),len(seed)))
    model_list = []

    # for i in range(len(layers)):
    #   for j in range(len(neurons)):
    #       abs_err_table[i,j], rel_err_table[i,j] = main_loop(layers[i], neurons[j], noises[0], BC="Dirichlet", prob="Direct")

    # np.savetxt('./abs_err_table.csv', abs_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    # np.savetxt('./rel_err_table.csv', rel_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')

    for i in range(len(noises)):
        for j in range(0, len(seed)):
            lambda_1_table1[i, j], time_table[i, j] =  main_loop(2, 5, noises[i], BC="Dirichlet", prob="Inverse", seed=seed[j])
    #     lambda_1_table2[i] =  main_loop(6, 15, noises[i], BC="Dirichlet", prob="Inverse")
    
    

    np.savetxt('./lambda_1_table1_51_point.csv', lambda_1_table1, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    np.savetxt('./time_table_51_point.csv', time_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    
#     lambda_1_log = []
#     for i in range(0, len(model_list)):
#         model_list[i].save_model(f"noise_0%_seed_{seed[i]}")
#         lambda_1_log.append(np.array(model_list[i].lambda_1_log)[:,0].flatten()[:,None])
    
#         np.savetxt(f'./lambda_1_log_noise_0%_seed_{seed[i]}.csv', lambda_1_log, delimiter=' ', fmt='%2.6f', newline=' \\\n')
        
#     for i in range(len(lambda_log)):

       

    #   np.savetxt('./lambda_1_table2.csv', lambda_1_table2, delimiter=' ', fmt='%2.6f', newline=' \\\n')



In [ ]:
# 201 n
def main_loop(layers, neurons, noise, BC, prob, seed):
    t = timeit.default_timer()
    print(noise)
    params = generate_param()
    print(seed)
    params["network"]["layers"] = [2] + layers*[neurons] + [1]
    params["data"]["noise"] = noise
    params["data"]["seed"] = seed
    loc_num = 4
    n = 201
    params["data"]["n_wall"] = n
    params["data"]["loc"] = np.linspace(0, 1, loc_num) #in %
    params["data"]["meu"] = [n]*loc_num

    # case = CasesChannel()
    case = CasesChannel(add_noise=True)
    case.generate_train_data(param=params, bc_type=BC)
    case.generate_test_data(param=params)
    case.plot()

    model = PinnVP(data=case.data, adaptive_model=False,
                 params=params, bc_type=BC, problem_type=prob)

    model.fit_newton()
    model_list.append(model)
    

    results = PostChannel(model=model,
                      params=params,
                      save_fig=False)

    results.display_loss()
    
#     lambda_log = []
    lambda_log = (np.array(model.lambda_1_log)[:,0])
    x = np.linspace(1, len(lambda_log), len(lambda_log))
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6), constrained_layout=True, dpi=300)
    ax.plot(x, lambda_log, "r", linestyle="solid", label="Noise free")
    ax.set_xlabel("Iterations", fontsize=12)
    ax.set_ylabel("Lambda Value", fontsize=12)
    ax.grid(linestyle="--")
    ax.legend(fontsize=10)
    ax.set_title('Lambda History', fontsize=17)
    ax.tick_params(axis='both', which='major', labelsize=10)


    #   case
    results.display_contour()
    elapsed = timeit.default_timer() - t

    if prob == "Direct":
        return results.abs_err_u, results.rel_err_u
    else:
        return model.lambda_1_log[-1][0][0], elapsed


# RUN
if __name__ == "__main__": 
    layers = [2] # num of hidden layers
    neurons = [5] # num of neurons
    noises = [0.00, 0.01, 0.03] #, 0.15] # noise level
    seed = [543212345, 2345, 5432123, 543212, 54321, 345, 543, 54, 5, 4] # num of hidden layers
    # N_u = [500, 1000, 1500, 2000] # num of training data 

#     abs_err_table = np.zeros((len(layers), len(neurons)))
#     rel_err_table = np.zeros((len(layers), len(neurons)))

    lambda_1_table1 = np.zeros((len(noises),len(seed)))
#     lambda_1_table2 = np.zeros(len(noises))
    time_table = np.zeros((len(noises),len(seed)))
    model_list = []

    # for i in range(len(layers)):
    #   for j in range(len(neurons)):
    #       abs_err_table[i,j], rel_err_table[i,j] = main_loop(layers[i], neurons[j], noises[0], BC="Dirichlet", prob="Direct")

    # np.savetxt('./abs_err_table.csv', abs_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    # np.savetxt('./rel_err_table.csv', rel_err_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')

    for i in range(len(noises)):
        for j in range(0, len(seed)):
            lambda_1_table1[i, j], time_table[i, j] =  main_loop(2, 5, noises[i], BC="Dirichlet", prob="Inverse", seed=seed[j])
    #     lambda_1_table2[i] =  main_loop(6, 15, noises[i], BC="Dirichlet", prob="Inverse")
    
    

    np.savetxt('./lambda_1_table1_201_point.csv', lambda_1_table1, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    np.savetxt('./time_table_201_point.csv', time_table, delimiter=' ', fmt='%2.6f', newline=' \\\n')
    
#     lambda_1_log = []
#     for i in range(0, len(model_list)):
#         model_list[i].save_model(f"noise_0%_seed_{seed[i]}")
#         lambda_1_log.append(np.array(model_list[i].lambda_1_log)[:,0].flatten()[:,None])
    
#         np.savetxt(f'./lambda_1_log_noise_0%_seed_{seed[i]}.csv', lambda_1_log, delimiter=' ', fmt='%2.6f', newline=' \\\n')
        
#     for i in range(len(lambda_log)):

       

    #   np.savetxt('./lambda_1_table2.csv', lambda_1_table2, delimiter=' ', fmt='%2.6f', newline=' \\\n')



In [ ]:
noise = noises
lambda_ = []
for i in noise:
    lambda_.append(pd.read_csv(f"lambda_1_log_noise_{i}.csv").to_numpy())
    

x0 = np.linspace(0, len(lambda_[0][:,0]), len(lambda_[0][:,0]))
x1 = np.linspace(0, len(lambda_[1][:,0]), len(lambda_[1][:,0]))
x2 = np.linspace(0, len(lambda_[2][:,0]), len(lambda_[2][:,0]))
x3 = np.linspace(0, len(lambda_[3][:,0]), len(lambda_[3][:,0]))


In [ ]:
# base = pd.read_csv("baseline_test.csv").to_numpy()

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 8), constrained_layout=True, dpi=300)
# ax.plot(base[:,0], base[:,1], "k", linestyle="solid", label="Baseline")
ax.plot(x0, lambda_[0][:,0], "r", linestyle="solid", label="Noise free")
ax.plot(x1, lambda_[1][:,0], "b", linestyle="dashed", label="5 percent noise")
ax.plot(x2, lambda_[2][:,0], "g", linestyle="dashdot", label="10 percent noise")
ax.plot(x3, lambda_[3][:,0], "c", linestyle="dotted", label="15 percent noise")
# ax.plot(M[4][:,0], M[4][:,1], "purple", linestyle="-.", label="M = 10000")

# ax.set_xlim(0, x1[-1])
# ax.set_yscale("log")
ax.set_xlabel("Iterations", fontsize=24)
ax.set_ylabel("$\lambda$", fontsize=24)
ax.grid(linestyle="--")
ax.legend(fontsize=20)
ax.set_title('$\lambda$ History', fontsize=35)
ax.tick_params(axis='both', which='major', labelsize=20)